Lectura de xml

In [1]:
import pandas as pd
import xml.etree.ElementTree as etree


tree = etree.parse("muni-fi-spr16.xml")
root = tree.getroot()

columns_problem = ["Name", "NrDays", "SlotsPerDay", "NrWeeks"]
columns_optimization = ["Time", "Room", "Distribution", "Student"]
columns_rooms = ["RoomId", "Capacity", "PreviousRoomIdList", "TravelTimeValueList", "UnavailableDaysList", "UnavailableStartList", "UnavailableLengthList", "UnavailableWeeksList"]
columns_courses = ["CourseId", "ConfigIdList"]
columns_configs = ["ConfigId", "SubpartIdList"]
columns_subparts = ["SubpartId", "ClassIdList"]
columns_classes = ["ClassId", "ClassLimit", "ClassParent"]
columns_distributions = ["DistributionId", "DistributionType", "DistributionRequired", "DistributionPenalty", "ClassesList"]
columns_students = ["StudentId", "CoursesList"]
columns_relation_class_room = ["ClassId", "RoomId", "RoomPenalty"]
columns_relation_class_time = ["ClassId", "TimeDays", "TimeStart", "TimeLength", "TimeWeeks", "TimePenalty"]

df_problem = pd.DataFrame(columns = columns_problem)
df_optimization = pd.DataFrame(columns = columns_optimization)
df_rooms = pd.DataFrame(columns = columns_rooms)
df_courses = pd.DataFrame(columns = columns_courses)
df_configs = pd.DataFrame(columns = columns_configs)
df_subparts = pd.DataFrame(columns = columns_subparts)
df_classes = pd.DataFrame(columns = columns_classes)
df_distributions = pd.DataFrame(columns = columns_distributions)
df_students = pd.DataFrame(columns = columns_students)
df_relation_class_room = pd.DataFrame(columns = columns_relation_class_room)
df_relation_class_time = pd.DataFrame(columns = columns_relation_class_time)




df_problem = df_problem.append({"Name": root.attrib.get("name"), "NrDays": root.attrib.get("nrDays"), "SlotsPerDay": root.attrib.get("slotsPerDay"), "NrWeeks": root.attrib.get("nrWeeks")}, ignore_index = True)

for node in root:
    if node.tag == "optimization":
        df_optimization = df_optimization.append({"Time": node.attrib.get("time"), "Room": node.attrib.get("room"), "Distribution": node.attrib.get("distribution"), "Student": node.attrib.get("student")}, ignore_index = True)
    
    elif node.tag == "rooms":
        for room in node:
            roomid = room.attrib.get("id")
            cap = room.attrib.get("capacity")
            prevroomlist = []
            valuelist = []
            dayslist = []
            startlist = []
            lengthlist = []
            weekslist = []
            if len(list(child.tag for child in room.iter() if child is not room)) > 0:
                for child in room:
                    if child.tag == "travel":
                        prevroomid = child.attrib.get("room")
                        value = child.attrib.get("value")
                        prevroomlist.append(prevroomid)
                        valuelist.append(value)
                    elif child.tag == "unavailable":
                        days = child.attrib.get("days")
                        start = child.attrib.get("start")
                        length = child.attrib.get("length")
                        weeks = child.attrib.get("weeks")
                        dayslist.append(days)
                        startlist.append(start)
                        lengthlist.append(length)
                        weekslist.append(weeks)
            if len(prevroomlist) == 0:
                prevroomlist = None
            if len(valuelist) == 0:
                valuelist = None
            if len(dayslist) == 0:
                dayslist = None
            if len(startlist) == 0:
                startlist = None
            if len(lengthlist) == 0:
                lengthlist = None
            if len(weekslist) == 0:
                weekslist = None
            df_rooms = df_rooms.append(pd.Series([roomid, cap, prevroomlist, valuelist, dayslist, startlist, lengthlist, weekslist], index = columns_rooms), ignore_index = True)
            
    elif node.tag == "courses":
        for course in node:
            #print(course.tag)
            courseid = course.attrib.get("id")
            configlist = []
            for config in course:
                #print(config.tag)
                configid = config.attrib.get("id")
                configlist.append(configid)
                subpartlist = []
                for subpart in config:
                    #print(subpart.tag)
                    subpartid = subpart.attrib.get("id")
                    subpartlist.append(subpartid)
                    claselist = []
                    for clase in subpart:
                        #print(clase.tag)
                        claseid = clase.attrib.get("id")
                        claselist.append(claseid)
                        claselimit = clase.attrib.get("limit")
                        claseparent = clase.attrib.get("parent") if clase.attrib.get("parent") is not None else None
                        df_classes = df_classes.append(pd.Series([claseid, claselimit, claseparent], index = columns_classes), ignore_index = True)
                        if len(list(child.tag for child in clase.iter() if child is not clase)) > 0:
                            for child in clase:
                                #print(child.tag)
                                if child.tag == "room":
                                    roomid = child.attrib.get("id")
                                    roompenalty = child.attrib.get("penalty")
                                    df_relation_class_room = df_relation_class_room.append(pd.Series([claseid, roomid, roompenalty], index = columns_relation_class_room), ignore_index = True)
                                elif child.tag == "time":
                                    timedays = child.attrib.get("days")
                                    timestart = child.attrib.get("start")
                                    timelength = child.attrib.get("length")
                                    timeweeks = child.attrib.get("weeks")
                                    timepenalty = child.attrib.get("penalty")
                                    df_relation_class_time = df_relation_class_time.append(pd.Series([claseid, timedays, timestart, timelength, timeweeks, timepenalty], index = columns_relation_class_time), ignore_index = True)
                    df_subparts = df_subparts.append(pd.Series([subpartid, claselist], index = columns_subparts), ignore_index = True)
                df_configs = df_configs.append(pd.Series([configid, subpartlist], index = columns_configs), ignore_index = True)
            df_courses = df_courses.append(pd.Series([courseid, configlist], index = columns_courses), ignore_index = True)
                                    
    elif node.tag == "distributions":
        contador = 1
        for distribution in node:
            #print(distribution.tag)
            distribid = contador
            distribtype = distribution.attrib.get("type")
            distribrequired = distribution.attrib.get("required") if distribution.attrib.get("required") is not None else None
            distribpenalty = distribution.attrib.get("penalty") if distribution.attrib.get("penalty") is not None else None
            listaclases = []
            for clase in distribution:
                #print(clase.tag)
                listaclases.append(clase.attrib.get("id"))
            df_distributions = df_distributions.append(pd.Series([distribid, distribtype, distribrequired, distribpenalty, listaclases], index = columns_distributions), ignore_index = True)
            contador += 1
             
    elif node.tag == "students":
        for student in node:
            studentid = student.attrib.get("id")
            listacourses = []
            for course in student:
                listacourses.append(course.attrib.get("id"))
            df_students = df_students.append(pd.Series([studentid, listacourses], index = columns_students), ignore_index = True)
            
            


print("################################################################################################################")
print("DataFrame Problem")
display(df_problem)
print("################################################################################################################")
print("DataFrame Optimization")
display(df_optimization)
print("################################################################################################################")
print("DataFrame Rooms")
display(df_rooms)
print("################################################################################################################")
print("DataFrame Courses")
display(df_courses)
print("################################################################################################################")
print("DataFrame Configs")
display(df_configs)
print("################################################################################################################")
print("DataFrame Subparts")
display(df_subparts)
print("################################################################################################################")
print("DataFrame Classes")
display(df_classes)
print("################################################################################################################")
print("DataFrame Distributions")
display(df_distributions)
print("################################################################################################################")
print("DataFrame Students")
display(df_students)
print("################################################################################################################")
print("DataFrame RelationClassRoom")
display(df_relation_class_room)
print("################################################################################################################")
print("DataFrame RelationClassTime")
display(df_relation_class_time)
print("################################################################################################################")

################################################################################################################
DataFrame Problem


,Name,NrDays,SlotsPerDay,NrWeeks
0,muni-fi-spr16,7,288,15


################################################################################################################
DataFrame Optimization


,Time,Room,Distribution,Student
0,3,1,10,5


################################################################################################################
DataFrame Rooms


...

################################################################################################################
DataFrame RelationClassRoom


,ClassId,RoomId,RoomPenalty
0,1,18,0
1,15,23,0
2,15,8,0
3,15,12,0
4,16,23,4
...,...,...,...
2404,526,27,4
2405,526,30,0
2406,526,6,0
2407,526,7,1


################################################################################################################
DataFrame RelationClassTime


,ClassId,TimeDays,TimeStart,TimeLength,TimeWeeks,TimePenalty
0,1,0000100,168,46,011110111111110,0
1,2,1000000,96,46,011111011111110,0
2,2,0000100,96,46,011110111111110,0
3,2,1000000,108,46,011111011111110,0
4,2,0000100,108,46,011110111111110,0
...,...,...,...,...,...,...
9551,571,1000000,181,18,010101010101010,0
9552,572,1000000,200,18,011111011111110,0
9553,573,0001000,96,18,010101010101010,0
9554,574,0001000,115,18,011111111111110,0


################################################################################################################


Analisis de datos

In [2]:
rest_duras = {}
rest_blandas = {}
for index, row in df_distributions.iterrows():
    if row["DistributionRequired"] == "true":
        if row["DistributionType"] not in rest_duras.keys():
            rest_duras[row["DistributionType"]] = 0
        rest_duras[row["DistributionType"]]+=1
    else:
        if row["DistributionType"] not in rest_blandas.keys():
            rest_blandas[row["DistributionType"]] = 0
        rest_blandas[row["DistributionType"]]+=1


            
print("Restricciones duras: ", rest_duras)
print("Restricciones blandas: ", rest_blandas)

Restricciones duras:  {'SameAttendees': 404, 'SameDays': 59, 'SameRoom': 25, 'WorkDay(24)': 15, 'WorkDay(48)': 11, 'DifferentDays': 8, 'Precedence': 73, 'WorkDay(36)': 1, 'NotOverlap': 9, 'SameTime': 30, 'SameWeeks': 1, 'MaxDayLoad(72)': 1, 'WorkDay(120)': 3, 'MaxDayLoad(60)': 1, 'WorkDay(108)': 1, 'MaxBlock(20,9)': 3}
Restricciones blandas:  {'Precedence': 26, 'SameDays': 10, 'WorkDay(36)': 1, 'WorkDay(48)': 5, 'WorkDay(72)': 1, 'DifferentDays': 7, 'SameRoom': 1, 'NotOverlap': 42, 'MaxBlock(20,9)': 1, 'MaxBlock(40,9)': 1}


In [3]:
#Analizar capacidad de clases

print("Capacidad de las clases | Cantidad de clases con esa capacidad")
display(df_classes.groupby("ClassLimit").count()["ClassId"])


Capacidad de las clases | Cantidad de clases con esa capacidad


ClassLimit
0       4
1      19
10     24
100     8
112     1
       ..
74      1
8       1
80      5
86      1
90      3
Name: ClassId, Length: 76, dtype: int64

In [4]:
print("Capacidad de las clases | Clases con esa capacidad")
dicc_capacidad_clase = {}
for index,row in df_classes.iterrows():
    if int(row["ClassLimit"]) not in dicc_capacidad_clase.keys():
        dicc_capacidad_clase[int(row["ClassLimit"])] = [int(row["ClassId"])]
    else:
        dicc_capacidad_clase[int(row["ClassLimit"])].append(int(row["ClassId"]))      
print(dicc_capacidad_clase)

Capacidad de las clases | Clases con esa capacidad
{20: [1, 5, 6, 29, 31, 76, 77, 78, 79, 81, 122, 123, 139, 140, 141, 204, 243, 244, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 377, 379, 386, 397, 425, 426, 427, 451, 452, 453, 454, 474, 505, 507, 521, 525, 526], 1: [2, 3, 4, 7, 10, 11, 12, 13, 14, 15, 16, 18, 20, 27, 28, 557, 570, 571, 575], 2: [8, 9, 534, 535, 536, 543, 544, 545, 572], 0: [17, 25, 26, 546], 70: [19, 21, 32, 34, 211, 449, 470], 80: [22, 74, 75, 217, 317], 10: [23, 24, 36, 207, 210, 225, 235, 312, 337, 338, 361, 362, 363, 364, 365, 366, 373, 375, 376, 391, 392, 475, 530, 547], 30: [30, 108, 109, 126, 127, 128, 190, 191, 192, 194, 196, 197, 198, 199, 246, 308, 357, 434, 446, 447, 463, 464, 465], 100: [33, 35, 41, 155, 324, 389, 393, 513], 15: [37, 43, 44, 124, 132, 133, 136, 142, 143, 200, 205, 214, 247, 248, 258, 260, 261, 262, 263, 264, 265, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 309, 310, 340, 34

In [5]:
#print(sorted(dicc_capacidad_clase.keys()))
# 253 clases con capacidad menor o igual a 20
# 245 clases con capacidad mayor a 20 y menor o igual a 50
# 44 clases con capacidad mayor a 50 y menor o igual a 100
# 33 clases con capacidad mayor a 100

contador = 0
for i in dicc_capacidad_clase:
    if i > 100:
        print(i, len(dicc_capacidad_clase[i]))
        contador += len(dicc_capacidad_clase[i])
#print(contador)

560 1
450 1
120 1
239 2
200 6
350 1
310 1
172 1
150 3
180 4
250 2
315 1
131 1
420 1
400 1
186 1
128 1
112 1
122 1
140 1
168 1


In [6]:
print("Capacidad | Cantidad de Salas | Salas")

dic_salas = {}
for index, row in df_rooms.iterrows():
    x = int(row["Capacity"])
    if x <= 20:
        if 1 not in dic_salas.keys():
            dic_salas[1] = [0,[]]
        dic_salas[1][1].append(row["RoomId"])
        dic_salas[1][0]+=1
    elif x > 20 and x <= 50:
        if 2 not in dic_salas.keys():
            dic_salas[2] = [0,[]]
        dic_salas[2][1].append(row["RoomId"])
        dic_salas[2][0]+=1
    elif x > 50 and x <= 100:
        if 3 not in dic_salas.keys():
            dic_salas[3] = [0,[]]
        dic_salas[3][1].append(row["RoomId"])
        dic_salas[3][0]+=1
    else:
        if 4 not in dic_salas.keys():
            dic_salas[4] = [0,[]]
        dic_salas[4][1].append(row["RoomId"])
        dic_salas[4][0]+=1
        
for i in sorted(dic_salas):
    print(i, dic_salas[i])


Capacidad | Cantidad de Salas | Salas
1 [16, ['1', '2', '4', '5', '9', '10', '11', '14', '15', '19', '20', '24', '28', '31', '32', '34']]
2 [11, ['3', '6', '7', '13', '17', '18', '21', '27', '29', '30', '35']]
3 [5, ['16', '22', '25', '26', '33']]
4 [3, ['8', '12', '23']]


In [7]:
grupo_clases_grandes = []
grupo_clases_medianas = []
grupo_clases_chicas = []
for i in dicc_capacidad_clase:
    if i > 50:
        for j in dicc_capacidad_clase[i]:
            grupo_clases_grandes.append(j)
    if i > 20 and i <= 50:
        for j in dicc_capacidad_clase[i]:
            grupo_clases_medianas.append(j)
    if i <= 20:
        for j in dicc_capacidad_clase[i]:
            grupo_clases_chicas.append(j)
print(grupo_clases_grandes)

[19, 21, 32, 34, 211, 449, 470, 22, 74, 75, 217, 317, 33, 35, 41, 155, 324, 389, 393, 513, 45, 50, 88, 107, 110, 111, 125, 226, 259, 129, 144, 293, 305, 327, 355, 367, 160, 175, 193, 201, 202, 212, 476, 208, 249, 339, 354, 213, 358, 227, 228, 229, 230, 231, 232, 233, 234, 424, 450, 458, 266, 288, 304, 306, 307, 412, 323, 378, 394, 410, 411, 421, 443, 444, 484, 504, 508]


In [8]:
dicc_clases_grandes = {}
dicc_clases_medianas = {}
dicc_clases_chicas = {}

    
for i in grupo_clases_grandes:
    dicc_clases_grandes[i] = i
    
for i in grupo_clases_medianas:
    dicc_clases_medianas[i] = i
    
for i in grupo_clases_chicas:
    dicc_clases_chicas[i] = i
print(dicc_clases_grandes)

{19: 19, 21: 21, 32: 32, 34: 34, 211: 211, 449: 449, 470: 470, 22: 22, 74: 74, 75: 75, 217: 217, 317: 317, 33: 33, 35: 35, 41: 41, 155: 155, 324: 324, 389: 389, 393: 393, 513: 513, 45: 45, 50: 50, 88: 88, 107: 107, 110: 110, 111: 111, 125: 125, 226: 226, 259: 259, 129: 129, 144: 144, 293: 293, 305: 305, 327: 327, 355: 355, 367: 367, 160: 160, 175: 175, 193: 193, 201: 201, 202: 202, 212: 212, 476: 476, 208: 208, 249: 249, 339: 339, 354: 354, 213: 213, 358: 358, 227: 227, 228: 228, 229: 229, 230: 230, 231: 231, 232: 232, 233: 233, 234: 234, 424: 424, 450: 450, 458: 458, 266: 266, 288: 288, 304: 304, 306: 306, 307: 307, 412: 412, 323: 323, 378: 378, 394: 394, 410: 410, 411: 411, 421: 421, 443: 443, 444: 444, 484: 484, 504: 504, 508: 508}


In [9]:
## Duracion en slots de clases:

df_relation_class_time["TimeLength"] = df_relation_class_time["TimeLength"].astype(int)
print("Promedio:")
display(df_relation_class_time["TimeLength"].mean())
print("Min:")
display(df_relation_class_time["TimeLength"].min())
print("Max:")
display(df_relation_class_time["TimeLength"].max())
print("Tiempo de inicio")
display(df_relation_class_time.groupby("TimeStart").count())
x = df_relation_class_time.groupby("TimeStart").count()
print("Count")
display(df_relation_class_time["TimeLength"].value_counts())
print("Mediana:")
display(df_relation_class_time["TimeLength"].median())
print("Cuartiles:")
display(df_relation_class_time["TimeLength"].quantile([0, 0.25, 0.5, 0.75, 1]))
print("Var:")
display(df_relation_class_time["TimeLength"].var())
print("Std:")
display(df_relation_class_time["TimeLength"].std())

Promedio:


22.235140226036

Min:


10

Max:


...

0.00    10.0
0.25    22.0
0.50    22.0
0.75    22.0
1.00    94.0
Name: TimeLength, dtype: float64

Var:


83.54083096934478

Std:


9.14006733943163

In [10]:
##SPFf=subpartes de la configuracion f
#Cantidad de subpartes de una configuración
dicc_SPFf = {}
for index, col in df_configs.iterrows():
    dicc_SPFf[col["ConfigId"]] = [int(x) for x in col["SubpartIdList"]]
print(dicc_SPFf)


{'1': [1], '2': [2], '3': [3], '4': [4], '5': [5, 6], '6': [7], '7': [8], '8': [9], '9': [10, 11], '10': [12], '11': [13], '12': [14], '13': [15], '14': [16], '15': [17], '16': [18, 19], '17': [20, 21], '18': [22], '19': [23], '20': [24], '21': [25], '22': [26], '23': [27], '24': [28], '25': [29], '26': [30], '27': [31], '28': [32], '29': [33], '30': [34, 35], '31': [36], '32': [37], '33': [38], '34': [39], '35': [40], '36': [41], '37': [42, 43], '38': [44, 45], '39': [46, 47, 48], '40': [49], '41': [50], '42': [51, 52], '43': [53], '44': [54], '45': [55, 56], '46': [57], '47': [58, 59], '48': [60, 61, 62], '49': [63], '50': [64], '51': [65], '52': [66, 67], '53': [68, 69], '54': [70], '55': [71], '56': [72], '57': [73], '58': [74], '59': [75], '60': [76], '61': [77, 78, 79], '62': [80, 81], '63': [82, 83], '64': [84, 85], '65': [86, 87], '66': [88, 89], '67': [90, 91], '68': [92, 93], '69': [94, 95, 96], '70': [97, 98], '71': [99], '72': [100], '73': [101], '74': [102], '75': [103], '

In [11]:
#DFe=configuraciones de cursos demandados por el estudiante e
#StudentId	CoursesList
dicc_DFe = {}
for index, col in df_students.iterrows():
    dicc_DFe[int(col["StudentId"])] = [int(x) for x in col["CoursesList"]]
print(dicc_DFe)

{1: [48, 52, 70, 119, 63], 2: [48, 52, 70, 119, 63], 3: [48, 70, 119], 4: [48, 52, 70, 119, 63], 5: [48, 52, 70, 119, 63], 6: [48, 52, 70, 119, 63], 7: [48, 52, 70, 63], 8: [63], 9: [52], 10: [52, 119, 63], 11: [98, 35, 37, 149, 53, 90, 173], 12: [98, 35, 36, 37, 149, 53, 90, 173], 13: [98, 82, 35, 36, 37, 149, 53, 90], 14: [98, 35, 36, 37, 149, 53, 90, 172], 15: [98, 35, 37, 149, 53, 90, 172, 30], 16: [98, 35, 37, 149, 53, 90, 173, 30], 17: [98, 35, 37, 149, 53, 152, 90, 172], 18: [98, 82, 35, 37, 149, 53, 90, 30], 19: [98, 35, 36, 37, 149, 53, 90, 173], 20: [98, 82, 35, 37, 149, 53, 90, 30], 21: [98, 82, 35, 37, 149, 53, 152, 90], 22: [98, 82, 35, 37, 149, 53, 152, 90], 23: [98, 35, 37, 149, 53, 152, 90, 173], 24: [98, 35, 37, 149, 53, 90, 172], 25: [98, 35, 37, 149, 53, 90, 172], 26: [25], 27: [64, 83, 53, 30], 28: [98, 101, 149, 53, 90], 29: [98, 149, 53, 90, 30], 30: [98, 149, 53, 90], 31: [98, 149, 53, 90, 30], 32: [98, 149, 53, 90, 30], 33: [98, 101, 149, 53, 90], 34: [98, 149, 

In [12]:
#UTs=tiempo no disponible para la sala s
#RoomId	UnavailableDaysList	UnavailableStartList	UnavailableLengthList	UnavailableWeeksList
dicc_UTs = {}
for index, col in df_rooms.iterrows():
    if col["UnavailableDaysList"] != None:
        dicc_UTs[col["RoomId"]] = [col["UnavailableDaysList"],col["UnavailableStartList"],col["UnavailableLengthList"],col["UnavailableWeeksList"]]
print(dicc_UTs)

{'3': [['0000100'], ['204'], ['36'], ['011110111111110']], '5': [['0000100', '0000100'], ['120', '144'], ['24', '96'], ['011110111111110', '011110111111110']], '6': [['0000100'], ['144'], ['96'], ['011110111111110']], '7': [['0000100'], ['144'], ['96'], ['011110111111110']], '8': [['0000100'], ['144'], ['96'], ['011110111111100']], '9': [['0000100', '0000100'], ['120', '144'], ['24', '96'], ['011110111111110', '011110111111110']], '10': [['0000100'], ['144'], ['96'], ['011110111111110']], '11': [['0000100'], ['144'], ['96'], ['011110111111110']], '12': [['0000100', '0000100'], ['144', '204'], ['30', '36'], ['011110111111100', '011110111111100']], '15': [['0000100'], ['144'], ['96'], ['011110111111110']], '16': [['0000100'], ['144'], ['96'], ['011110111111110']], '17': [['0000100'], ['144'], ['96'], ['011110111111110']], '18': [['0000100', '0000100'], ['144', '216'], ['22', '24'], ['011110111111110', '011110111111110']], '21': [['0000100'], ['144'], ['96'], ['011110111111110']], '22': [

In [13]:
## TDc=tiempos en los que se puede impartir la clase c
dicc_TDc = {}
for index,row in df_relation_class_time.iterrows():
    if int(row["ClassId"]) not in dicc_TDc.keys():
        dicc_TDc[int(row["ClassId"])] = [[row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]]]
    else:
        dicc_TDc[int(row["ClassId"])].append([row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]])

#for i in dicc_TDc:
    #print(i,dicc_TDc[i])

dicc_TDc2 = dicc_TDc
dicc_TDc = {}
contador = 0
for e,i in enumerate(dicc_TDc2):
    for j in dicc_TDc2[i]:
        if i not in dicc_TDc.keys():
            dicc_TDc[i] = []
        dicc_TDc[i].append(contador)
        contador+=1
#cont2 = 5000
#for i in dicc_TDc:
#    dicc_TDc[i].append(cont2)
#    cont2 += 1
print(dicc_TDc)


{1: [0], 2: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18], 3: [19], 4: [20], 5: [21], 6: [22], 7: [23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], 8: [49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76], 9: [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104], 10: [105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148], 11: [149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192], 12: [193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 20

In [14]:
dicc_t = {}
for index,row in df_relation_class_time.iterrows():
    dicc_t[index] = [row["ClassId"],row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]]
print(dicc_t)

{0: ['1', '0000100', '168', 46, '011110111111110', '0'], 1: ['2', '1000000', '96', 46, '011111011111110', '0'], 2: ['2', '0000100', '96', 46, '011110111111110', '0'], 3: ['2', '1000000', '108', 46, '011111011111110', '0'], 4: ['2', '0000100', '108', 46, '011110111111110', '0'], 5: ['2', '1000000', '120', 46, '011111011111110', '0'], 6: ['2', '0000100', '120', 46, '011110111111110', '0'], 7: ['2', '1000000', '132', 46, '011111011111110', '0'], 8: ['2', '0000100', '132', 46, '011110111111110', '2'], 9: ['2', '1000000', '144', 46, '011111011111110', '0'], 10: ['2', '0000100', '144', 46, '011110111111110', '2'], 11: ['2', '1000000', '156', 46, '011111011111110', '0'], 12: ['2', '0000100', '156', 46, '011110111111110', '2'], 13: ['2', '1000000', '168', 46, '011111011111110', '0'], 14: ['2', '0000100', '168', 46, '011110111111110', '2'], 15: ['2', '1000000', '180', 46, '011111011111110', '2'], 16: ['2', '0000100', '180', 46, '011110111111110', '2'], 17: ['2', '1000000', '192', 46, '011111011

In [15]:
## SDc=salas disponibles para la clase c

dicc_SDispc = {}
for index, row in df_relation_class_room.iterrows():
    if int(row["ClassId"]) not in dicc_SDispc.keys():
        dicc_SDispc[int(row["ClassId"])]=[int(row["RoomId"])]
    else:
        dicc_SDispc[int(row["ClassId"])].append(int(row["RoomId"]))

print(dicc_SDispc)

dicc_SDispc2 = {}
for e in range(1,len(df_classes)+1):
    for i in dicc_SDispc:
        #print(i,e,dicc_SRc[i])
        if str(i) == str(e):
            dicc_SDispc2[e] = dicc_SDispc[i]
        if e not in dicc_SDispc2.keys():
            dicc_SDispc2[e] = []
print(dicc_SDispc2)

{1: [18], 15: [23, 8, 12], 16: [23, 8, 12], 19: [8], 21: [22], 22: [23, 8, 12], 25: [23, 12], 26: [9, 5], 27: [22], 28: [23, 8, 12], 29: [16, 21, 22, 3, 13], 30: [21, 3, 13, 25, 27, 7, 17], 31: [16, 21, 22, 3, 13, 25, 27, 30, 6, 7, 17], 32: [16, 22, 23, 8, 12], 33: [16, 22, 23, 8, 12], 34: [8], 35: [16, 22, 23, 8, 12], 36: [30], 37: [16, 21, 22, 3, 13, 25, 27, 30], 38: [27], 39: [27], 40: [16, 21, 22, 3, 13, 25, 27, 7, 17], 41: [8], 42: [16, 21, 22, 3, 13, 25, 27, 7, 17], 43: [30], 44: [2], 45: [16, 21, 22, 3, 13, 25], 46: [16, 21, 22, 3, 13, 25, 27, 30], 47: [16, 21, 22, 3, 13, 25, 27, 23, 8, 12], 48: [15], 49: [15], 50: [23, 12], 51: [16, 21, 22, 3, 13, 25, 27, 7, 17], 52: [16, 21, 22, 3, 13, 25, 27, 7], 53: [16, 21, 22, 3, 13, 25, 27, 7, 17], 54: [16, 21, 22, 3, 13, 25, 27, 7, 17], 55: [16, 21, 22, 3, 13, 25, 27, 7, 17], 56: [16, 21, 22, 3, 13, 25, 27, 7, 17], 57: [16, 21, 22, 3, 13, 25, 27, 7, 17], 58: [16, 21, 22, 3, 13, 25, 27, 7, 17], 59: [16, 21, 22, 3, 13, 25, 27, 7, 17], 60: 

{1: [18], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [23, 8, 12], 16: [23, 8, 12], 17: [], 18: [], 19: [8], 20: [], 21: [22], 22: [23, 8, 12], 23: [], 24: [], 25: [23, 12], 26: [9, 5], 27: [22], 28: [23, 8, 12], 29: [16, 21, 22, 3, 13], 30: [21, 3, 13, 25, 27, 7, 17], 31: [16, 21, 22, 3, 13, 25, 27, 30, 6, 7, 17], 32: [16, 22, 23, 8, 12], 33: [16, 22, 23, 8, 12], 34: [8], 35: [16, 22, 23, 8, 12], 36: [30], 37: [16, 21, 22, 3, 13, 25, 27, 30], 38: [27], 39: [27], 40: [16, 21, 22, 3, 13, 25, 27, 7, 17], 41: [8], 42: [16, 21, 22, 3, 13, 25, 27, 7, 17], 43: [30], 44: [2], 45: [16, 21, 22, 3, 13, 25], 46: [16, 21, 22, 3, 13, 25, 27, 30], 47: [16, 21, 22, 3, 13, 25, 27, 23, 8, 12], 48: [15], 49: [15], 50: [23, 12], 51: [16, 21, 22, 3, 13, 25, 27, 7, 17], 52: [16, 21, 22, 3, 13, 25, 27, 7], 53: [16, 21, 22, 3, 13, 25, 27, 7, 17], 54: [16, 21, 22, 3, 13, 25, 27, 7, 17], 55: [16, 21, 22, 3, 13, 25, 27, 7, 17], 56: [16, 21, 22, 3, 13, 25, 

In [16]:
#SAc=clases c de la distribución same attendees
dicc_SAc = {}
for index, col in df_distributions.iterrows():
    if col["DistributionType"] == "SameAttendees" and col["DistributionRequired"] == "true":
        dicc_SAc[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 

print(dicc_SAc)

dicc_SAc_B = {}
for index, col in df_distributions.iterrows():
    if col["DistributionType"] == "SameAttendees" and col["DistributionRequired"] == None:
        dicc_SAc_B[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 

print(dicc_SAc_B)

{2: [2, 4, 80, 81, 315, 316], 3: [3, 29], 4: [4, 202, 217, 239], 8: [8, 9, 80, 81], 17: [17, 481, 482], 18: [18, 31], 19: [19, 410, 455, 456], 20: [20, 236, 237, 238], 23: [23, 24, 434, 435, 436], 32: [32, 49], 39: [39, 41, 135], 40: [40, 39], 42: [42, 68, 207, 308, 309, 310, 474], 44: [44, 83, 88, 325], 45: [45, 83, 137], 51: [51, 50], 55: [55, 50], 58: [58, 50], 63: [63, 50], 70: [70, 50], 72: [72, 50], 89: [89, 88], 93: [93, 88], 95: [95, 88], 99: [99, 88], 101: [101, 88], 103: [103, 88], 116: [116, 110], 118: [118, 110], 120: [120, 110], 142: [142, 143], 153: [153, 154], 155: [155, 156], 157: [157, 155], 160: [160, 161, 162, 187], 163: [163, 160], 165: [165, 160], 167: [167, 160], 170: [170, 160], 173: [173, 160], 176: [176, 175], 179: [179, 175], 181: [181, 175], 183: [183, 175], 185: [185, 175], 188: [188, 187], 190: [190, 191, 192], 196: [196, 193], 198: [198, 193], 212: [212, 220, 304, 327], 215: [215, 260, 261, 266, 267, 268, 293, 294, 295, 372, 442], 220: [220, 221, 239, 240,

In [17]:
#SAc=clases c de la distribución same attendees
for index, col in df_distributions.iterrows():
    print(col["DistributionType"])


SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAttendees
SameAt

In [18]:
#SSc =clases c de la distribución same start
dicc_SSc = {}
for index, col in df_distributions.iterrows():
    #print(col["DistributionType"])
    if col["DistributionType"] == "SameStart" and col["DistributionRequired"]=="true":
        dicc_SSc[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 
print(dicc_SSc)

dicc_SSc_B = {}
for index, col in df_distributions.iterrows():
    #print(col["DistributionType"])
    if col["DistributionType"] == "SameStart" and col["DistributionRequired"]==None:
        dicc_SSc_B[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 
print(dicc_SSc_B)

{}
{}


In [19]:
#STc =clases c de la distribución same time
dicc_STc = {}
for index, col in df_distributions.iterrows():
    #print(col["DistributionType"])
    if col["DistributionType"] == "SameTime" and col["DistributionRequired"]=="true":
        dicc_STc[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 
#print(dicc_STc)

dicc_STc_B = {}
for index, col in df_distributions.iterrows():
    #print(col["DistributionType"])
    if col["DistributionType"] == "SameTime" and col["DistributionRequired"]==None:
        dicc_STc_B[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 
#print(dicc_STc_B)
dicc_STc2 = dicc_STc.copy()
for e in range(len(df_classes)+1):
    for i in dicc_STc2:
        #print(i,e,dicc_SRc[i])
        if str(i) == str(e):
            dicc_STc[e] = dicc_STc2[i]
        if e not in dicc_STc.keys():
            dicc_STc[e] = []
print(dicc_STc)

{340: [340, 341], 342: [342, 343], 344: [344, 345], 346: [346, 347], 348: [348, 349], 350: [350, 351], 459: [459, 460], 461: [461, 462], 477: [477, 478], 479: [479, 480], 481: [481, 482], 84: [84, 85], 237: [237, 238], 247: [247, 248], 18: [18, 391], 17: [17, 374], 20: [20, 465], 10: [10, 318], 11: [11, 319], 12: [12, 320], 13: [13, 321], 14: [14, 322], 7: [7, 211], 250: [250, 251], 252: [252, 253], 254: [254, 255], 256: [256, 257], 8: [8, 220], 9: [9, 221], 2: [2, 31], 0: [], 1: [], 3: [], 4: [], 5: [], 6: [], 15: [], 16: [], 19: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78:

In [20]:
#SRc =clases c de la distribución same room
dicc_SRc = {}
for index, col in df_distributions.iterrows():
    if col["DistributionType"] == "SameRoom" and col["DistributionRequired"]=="true":
        dicc_SRc[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 
#print(dicc_SRc)

dicc_SRc_B = {}
for index, col in df_distributions.iterrows():
    if col["DistributionType"] == "SameRoom" and col["DistributionRequired"]== None:
        dicc_SRc_B[int(col["ClassesList"][0])] = [int(x) for x in col["ClassesList"]] 
#print(dicc_SRc_B)

dicc_SRc2 = {}
for e in range(len(df_classes)+1):
    for i in dicc_SRc:
        #print(i,e,dicc_SRc[i])
        if str(i) == str(e):
            dicc_SRc2[e] = dicc_SRc[i]
        if e not in dicc_SRc2.keys():
            dicc_SRc2[e] = []
dicc_SRc = dicc_SRc2
print(dicc_SRc)

{0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: [], 12: [], 13: [], 14: [], 15: [], 16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: [], 24: [], 25: [], 26: [], 27: [], 28: [], 29: [], 30: [], 31: [], 32: [], 33: [], 34: [], 35: [], 36: [], 37: [], 38: [], 39: [], 40: [], 41: [], 42: [], 43: [], 44: [], 45: [], 46: [], 47: [], 48: [], 49: [], 50: [], 51: [], 52: [], 53: [], 54: [], 55: [], 56: [], 57: [], 58: [], 59: [], 60: [], 61: [], 62: [], 63: [], 64: [], 65: [], 66: [], 67: [], 68: [], 69: [], 70: [], 71: [], 72: [], 73: [], 74: [], 75: [], 76: [], 77: [], 78: [], 79: [], 80: [], 81: [], 82: [], 83: [], 84: [84, 85], 85: [], 86: [], 87: [], 88: [], 89: [], 90: [], 91: [], 92: [], 93: [], 94: [], 95: [], 96: [], 97: [], 98: [], 99: [], 100: [], 101: [], 102: [], 103: [], 104: [], 105: [], 106: [], 107: [], 108: [], 109: [], 110: [], 111: [], 112: [112, 113], 113: [], 114: [114, 115], 115: [], 116: [116, 117], 117: [], 118: [118, 119], 

In [21]:
## SDc=clases c de la distribución same days
dicc_SDc = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays" and row["DistributionRequired"]=="true":
        c1 = row["ClassesList"][0]
        #print(c1)
        dicc_SDc[c1] = []
        for i in range(len(row["ClassesList"])):
            dicc_SDc[c1].append(row["ClassesList"][i])

dicc_SDc2 = dicc_SDc.copy()
for e in range(len(df_classes)+1):
    for i in dicc_SDc2:
        #print(i,e,dicc_SRc[i])
        if str(i) == str(e):
            dicc_SDc[e] = dicc_SDc2[i]
        if e not in dicc_SDc.keys():
            dicc_SDc[e] = []
print(dicc_SDc)

dicc_SDc_B = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="SameDays" and row["DistributionRequired"]==None:
        c1 = row["ClassesList"][0]
        #print(c1)
        dicc_SDc_B[c1] = []
        for i in range(len(row["ClassesList"])):
            dicc_SDc_B[c1].append(row["ClassesList"][i])
#print(dicc_SDc_B)

{'318': ['318', '319'], '320': ['320', '321'], '112': ['112', '113'], '114': ['114', '115'], '116': ['116', '117'], '118': ['118', '119'], '120': ['120', '121'], '472': ['472', '471'], '289': ['289', '290'], '291': ['291', '292'], '218': ['218', '219'], '243': ['243', '244'], '294': ['294', '295'], '296': ['296', '297'], '298': ['298', '299'], '300': ['300', '301'], '302': ['302', '303'], '335': ['335', '336'], '359': ['359', '360'], '421': ['421', '422', '423'], '463': ['463', '464'], '516': ['516', '517'], '328': ['328', '326'], '390': ['390', '526'], '216': ['216', '249'], '103': ['103', '104'], '340': ['340', '341'], '342': ['342', '343'], '344': ['344', '345'], '346': ['346', '347'], '348': ['348', '349'], '350': ['350', '351'], '459': ['459', '460'], '461': ['461', '462'], '477': ['477', '478'], '479': ['479', '480'], '481': ['481', '482'], '84': ['84', '85'], '237': ['237', '238'], '247': ['247', '248'], '18': ['18', '391'], '17': ['17', '374'], '20': ['20', '465'], '10': ['10',

In [22]:
## PRc=clases c de la distribución precedence
dicc_PRc = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence" and row["DistributionRequired"]=="true":
        c1 = row["ClassesList"][0]
        #print(c1)
        dicc_PRc[c1] = []
        for i in range(len(row["ClassesList"])):
            dicc_PRc[c1].append(row["ClassesList"][i])
print(dicc_PRc)

dicc_PRc_B = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"]=="Precedence" and row["DistributionRequired"]==None:
        c1 = row["ClassesList"][0]
        #print(c1)
        dicc_PRc_B[c1] = []
        for i in range(len(row["ClassesList"])):
            dicc_PRc_B[c1].append(row["ClassesList"][i])
print(dicc_PRc_B)

{'218': ['218', '219'], '243': ['243', '244'], '294': ['294', '295'], '296': ['296', '297'], '298': ['298', '299'], '300': ['300', '301'], '302': ['302', '303'], '335': ['335', '336'], '359': ['359', '360'], '421': ['421', '422', '423'], '463': ['463', '464'], '516': ['516', '517'], '222': ['222', '223'], '266': ['266', '287'], '288': ['288', '292'], '367': ['367', '372'], '74': ['74', '79'], '110': ['110', '111', '121'], '339': ['339', '351'], '246': ['246', '248'], '155': ['155', '156'], '153': ['153', '154']}
{'226': ['226', '359'], '129': ['129', '131'], '259': ['259', '265'], '293': ['293', '303'], '308': ['308', '309'], '458': ['458', '462'], '236': ['236', '238']}


In [23]:
#SPCc=id de la subparte a la que pertenece c
#dicc_subparts = {}
#dicc_SPCc = {}
#for index, row in df_subparts.iterrows():
#    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]

#for subpartes in dicc_subparts:
#    for clases in dicc_subparts[subpartes]:
#        dicc_SPCc[clases] = int(subpartes)
#print(dicc_SPCc)

In [24]:
#Dc=duración de slots de clase c
dicc_Dc = {}
for index, row in df_relation_class_time.iterrows():
    dicc_Dc[int(row["ClassId"])] = int(row["TimeLength"])
print(dicc_Dc)

{1: 46, 2: 46, 3: 22, 4: 46, 5: 39, 6: 39, 7: 22, 8: 22, 9: 22, 10: 10, 11: 10, 12: 10, 13: 10, 14: 10, 15: 50, 16: 22, 17: 22, 18: 22, 19: 10, 20: 22, 21: 22, 22: 22, 23: 22, 24: 22, 25: 30, 26: 22, 27: 72, 28: 22, 29: 22, 30: 22, 31: 46, 32: 22, 33: 34, 34: 22, 35: 22, 36: 22, 37: 22, 38: 22, 39: 22, 40: 22, 41: 22, 42: 22, 43: 22, 44: 22, 45: 22, 46: 22, 47: 22, 48: 22, 49: 22, 50: 22, 51: 22, 52: 22, 53: 22, 54: 22, 55: 22, 56: 22, 57: 22, 58: 22, 59: 22, 60: 22, 61: 22, 62: 22, 63: 22, 64: 22, 65: 22, 66: 22, 67: 22, 68: 22, 69: 22, 70: 22, 71: 22, 72: 22, 73: 22, 74: 22, 75: 22, 76: 22, 77: 22, 78: 22, 79: 22, 80: 22, 81: 22, 82: 22, 83: 22, 84: 22, 85: 22, 86: 22, 87: 34, 88: 22, 89: 22, 90: 22, 91: 22, 92: 22, 93: 22, 94: 22, 95: 22, 96: 22, 97: 22, 98: 22, 99: 22, 100: 22, 101: 22, 102: 22, 103: 22, 104: 22, 105: 22, 106: 22, 107: 22, 108: 22, 109: 22, 110: 22, 111: 22, 112: 10, 113: 10, 114: 10, 115: 10, 116: 10, 117: 10, 118: 10, 119: 10, 120: 10, 121: 10, 122: 22, 123: 22, 

In [25]:
#TSc=slot de inicio de la clase c
dicc_TSc = {}
for index, row in df_relation_class_time.iterrows():
    if int(row["ClassId"]) not in dicc_TSc:
        dicc_TSc[int(row["ClassId"])] = []
    if int(row["TimeStart"]) not in dicc_TSc[int(row["ClassId"])]:
        dicc_TSc[int(row["ClassId"])].append(int(row["TimeStart"]))

for i in dicc_TSc:
    dicc_TSc[i] = [int(x) for x in dicc_TSc[i]]
print(dicc_TSc)
    

{1: [168], 2: [96, 108, 120, 132, 144, 156, 168, 180, 192], 3: [168], 4: [144], 5: [196], 6: [196], 7: [96, 120, 144, 168, 192, 216], 8: [96, 120, 144, 168, 192, 216], 9: [96, 120, 144, 168, 192, 216], 10: [96, 108, 120, 132, 144, 156, 168, 180, 192, 204], 11: [96, 108, 120, 132, 144, 156, 168, 180, 192, 204], 12: [96, 108, 120, 132, 144, 156, 168, 180, 192, 204], 13: [96, 108, 120, 132, 144, 156, 168, 180, 192, 204], 14: [96, 108, 120, 132, 144, 156, 168, 180, 192, 204], 15: [204], 16: [216], 17: [96, 120, 144, 168, 192, 216], 18: [168], 19: [156], 20: [96, 120, 144, 168, 192, 216], 21: [216], 22: [144, 168, 192], 23: [168], 24: [144], 25: [174], 26: [96], 27: [192], 28: [216], 29: [120], 30: [168], 31: [96, 108, 120, 132, 144, 156, 168, 180, 192], 32: [96, 120, 144, 168, 192], 33: [120, 144, 156, 168, 180, 192, 204], 34: [120, 144, 168, 192], 35: [120, 144, 168, 192, 216], 36: [120], 37: [120, 144, 168, 192], 38: [96, 120, 144, 168, 192], 39: [120], 40: [96, 120, 144, 168, 192, 216],

In [26]:
## CPc=clase padre de la clase c
dicc_CPc = {}
for index,row in df_classes.iterrows():
    dicc_CPc[int(row["ClassId"])] = row["ClassParent"]
print(dicc_CPc)

{1: None, 2: None, 3: None, 4: None, 5: None, 6: None, 7: None, 8: None, 9: None, 10: None, 11: None, 12: None, 13: None, 14: None, 15: None, 16: None, 17: None, 18: None, 19: None, 20: None, 21: None, 22: None, 23: None, 24: '23', 25: None, 26: None, 27: None, 28: None, 29: None, 30: None, 31: None, 32: None, 33: None, 34: None, 35: None, 36: None, 37: None, 38: None, 39: None, 40: '39', 41: None, 42: None, 43: None, 44: None, 45: None, 46: None, 47: None, 48: None, 49: None, 50: None, 51: None, 52: None, 53: None, 54: None, 55: None, 56: None, 57: None, 58: None, 59: None, 60: None, 61: None, 62: None, 63: None, 64: None, 65: None, 66: None, 67: None, 68: None, 69: None, 70: None, 71: None, 72: None, 73: None, 74: None, 75: '74', 76: None, 77: None, 78: None, 79: None, 80: None, 81: None, 82: None, 83: None, 84: None, 85: None, 86: None, 87: None, 88: None, 89: None, 90: None, 91: None, 92: None, 93: None, 94: None, 95: None, 96: None, 97: None, 98: None, 99: None, 100: None, 101: No

In [27]:
## QSs=capacidad de estudiantes de la sala s 
dicc_QSs = {}
for index,row in df_rooms.iterrows():
    dicc_QSs[int(row["RoomId"])] = int(row["Capacity"])
print(dicc_QSs)

{1: 15, 2: 10, 3: 40, 4: 12, 5: 15, 6: 24, 7: 34, 8: 122, 9: 15, 10: 18, 11: 15, 12: 179, 13: 40, 14: 10, 15: 18, 16: 76, 17: 34, 18: 25, 19: 12, 20: 12, 21: 40, 22: 76, 23: 248, 24: 15, 25: 54, 26: 51, 27: 38, 28: 12, 29: 40, 30: 24, 31: 12, 32: 12, 33: 100, 34: 10, 35: 24}


In [28]:
## QCc=capacidad de estudiantes de la clase c
dicc_QCc = {}
for index,row in df_classes.iterrows():
    #if int(row["ClassId"]) not in dicc_QCc.keys():
        dicc_QCc[int(row["ClassId"])] = int(row["ClassLimit"])
print(dicc_QCc)

{1: 20, 2: 1, 3: 1, 4: 1, 5: 20, 6: 20, 7: 1, 8: 2, 9: 2, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 0, 18: 1, 19: 70, 20: 1, 21: 70, 22: 80, 23: 10, 24: 10, 25: 0, 26: 0, 27: 1, 28: 1, 29: 20, 30: 30, 31: 20, 32: 70, 33: 100, 34: 70, 35: 100, 36: 10, 37: 15, 38: 25, 39: 34, 40: 34, 41: 100, 42: 25, 43: 15, 44: 15, 45: 66, 46: 18, 47: 48, 48: 25, 49: 23, 50: 560, 51: 26, 52: 25, 53: 25, 54: 25, 55: 25, 56: 25, 57: 25, 58: 25, 59: 26, 60: 25, 61: 25, 62: 25, 63: 25, 64: 25, 65: 25, 66: 25, 67: 26, 68: 25, 69: 26, 70: 25, 71: 25, 72: 25, 73: 25, 74: 80, 75: 80, 76: 20, 77: 20, 78: 20, 79: 20, 80: 19, 81: 20, 82: 40, 83: 36, 84: 18, 85: 18, 86: 50, 87: 40, 88: 450, 89: 25, 90: 25, 91: 25, 92: 25, 93: 25, 94: 25, 95: 25, 96: 25, 97: 25, 98: 25, 99: 25, 100: 25, 101: 25, 102: 25, 103: 25, 104: 25, 105: 25, 106: 25, 107: 120, 108: 30, 109: 30, 110: 239, 111: 239, 112: 24, 113: 26, 114: 25, 115: 25, 116: 25, 117: 25, 118: 26, 119: 23, 120: 25, 121: 25, 122: 20, 123: 20, 124: 15, 125

In [29]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_lim = {}
dicc_curso_clase = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            #print(dicc_configs[j])
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    #print(dicc_subparts[k])
                    for l in dicc_subparts[k]:
                        #print(l)
                        #Agregué estas lineas para hacerme un diccionario de curso con clase#
                        if l not in dicc_curso_clase:
                            dicc_curso_clase[l] = []
                        dicc_curso_clase[l].append(p)
                        ######################################################################
                        if l in dicc_classes.keys():
                            n = int(dicc_classes[l])
                            if p not in dicc_lim.keys():
                                dicc_lim[p] = [n]
                            else:
                                dicc_lim[p].append(n)
                        

In [30]:
## cursos más demandados por estudiantes
dic_demanda_cursos = {}
for index,row in df_students.iterrows():
    for i in row["StudentId"]:
        for j in row["CoursesList"]:
            if j not in dic_demanda_cursos.keys():
                dic_demanda_cursos[j] = 1
                #print("agregado:",j,dic_demanda_cursos[j])
            else:
                dic_demanda_cursos[j] += 1
                #print("suma +1:",j,dic_demanda_cursos[j])
#print("Id Curso | Cantidad demandada")
for w in dic_demanda_cursos.keys():    
    print(w,dic_demanda_cursos[w])

48 763
52 268
70 449
119 108
63 264
98 134
35 205
37 138
149 190
53 209
90 123
173 176
36 55
82 151
172 94
30 56
152 92
25 192
64 69
83 189
101 67
96 9
147 24
220 14
93 9
91 57
92 69
67 1031
107 567
110 1019
50 69
163 151
84 612
164 95
94 239
142 40
38 1823
151 430
153 140
176 305
29 62
86 28
88 209
26 84
46 368
178 195
66 1120
117 885
181 890
105 911
45 1182
109 525
193 18
130 84
132 336
102 447
187 358
157 310
191 119
196 161
185 36
190 95
216 22
129 458
79 459
97 145
100 27
133 9
150 142
41 46
186 43
23 190
73 317
72 174
106 183
189 73
55 16
39 193
121 500
138 611
143 627
128 15
134 21
166 91
40 129
44 121
111 211
115 240
56 162
124 126
131 360
104 210
183 291
135 444
144 331
145 70
116 185
69 66
154 149
204 31
57 31
108 435
207 83
162 362
42 86
43 21
65 196
49 68
31 146
99 39
209 70
112 69
47 85
177 157
118 61
24 128
120 53
103 18
76 26
182 64
168 156
200 49
146 5
87 36
89 56
148 175
54 16
28 19
74 51
80 37
224 16
205 23
221 9
194 99
169 46
171 19
27 25
75 48
167 10
95 152
155 151


In [31]:
## Conteo de las salas mas utilizadas

dicc_conteo_clases_sala = {}
for index, row in df_relation_class_room.iterrows():
    if row["RoomId"] not in dicc_conteo_clases_sala.keys():
        dicc_conteo_clases_sala[row["RoomId"]] = 1
    else:
        dicc_conteo_clases_sala[row["RoomId"]] += 1
#print(dicc_conteo_clases_sala)
#print(len(dicc_conteo_clases_sala.keys()))
print("Sala | Cantidad de veces que se solicita")
#{k: v for k, v in sorted(dicc_conteo_clases_sala.items(), key=lambda item: item[1])}
#for i in sorted(dicc_conteo_clasess_sala)

Sala | Cantidad de veces que se solicita


In [32]:
## Dj=distribuciones de tipo j
#dicc_Dj = {1:"Same Attendees",2:"Same Start",3:"Same Room",4:"Same Time",5:"Same Days",6:"Precedence"}
dicc_Dj = {1:1, 2:2, 3:3, 4:4, 5:5, 6:6}

In [33]:
lista_I = []
for i in range(1,103):
    lista_I.append(i)
print(lista_I)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102]


In [34]:
dicc_Ij = {1:[],2:[],3:[],4:[],5:[],6:[]}
for index, row in df_distributions.iterrows():
    if row["DistributionType"] == "SameAttendees":
        dicc_Ij[1].append(row["DistributionId"])
    elif row["DistributionType"] == "SameStart":
        dicc_Ij[2].append(row["DistributionId"])
    elif row["DistributionType"] == "SameRoom":
        dicc_Ij[3].append(row["DistributionId"])
    elif row["DistributionType"] == "SameTime":
        dicc_Ij[4].append(row["DistributionId"])
    elif row["DistributionType"] == "SameDays":
        dicc_Ij[5].append(row["DistributionId"])
    elif row["DistributionType"] == "Precedence":
        dicc_Ij[6].append(row["DistributionId"])
        
print(dicc_Ij)

{1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475, 476, 477, 478, 479, 480, 481, 482, 483, 484, 485, 486, 487, 488, 489, 490, 491, 492, 493, 494, 495, 496, 497, 498, 499, 500, 501, 502, 503, 504, 505, 506, 507, 508, 509, 510

In [35]:
dicc_PTct = {}
for index, row in df_relation_class_time.iterrows():
    llave = (int(row["ClassId"]),int(index))
    dicc_PTct[llave] = int(row["TimePenalty"]) 

#dicc_PTct2 = dicc_PTct.copy()
#contador = 5000
#for i in dicc_PTct2:
#    llave = (i[0],contador)
#    dicc_PTct[llave] = 9999
#    contador += 1
print(dicc_PTct)

{(1, 0): 0, (2, 1): 0, (2, 2): 0, (2, 3): 0, (2, 4): 0, (2, 5): 0, (2, 6): 0, (2, 7): 0, (2, 8): 2, (2, 9): 0, (2, 10): 2, (2, 11): 0, (2, 12): 2, (2, 13): 0, (2, 14): 2, (2, 15): 2, (2, 16): 2, (2, 17): 2, (2, 18): 2, (3, 19): 0, (4, 20): 0, (5, 21): 0, (6, 22): 0, (7, 23): 0, (7, 24): 0, (7, 25): 0, (7, 26): 0, (7, 27): 0, (7, 28): 0, (7, 29): 0, (7, 30): 0, (7, 31): 0, (7, 32): 0, (7, 33): 10, (7, 34): 10, (7, 35): 10, (7, 36): 10, (7, 37): 10, (7, 38): 10, (7, 39): 10, (7, 40): 10, (7, 41): 10, (7, 42): 12, (7, 43): 10, (7, 44): 10, (7, 45): 12, (7, 46): 12, (7, 47): 12, (7, 48): 12, (8, 49): 6, (8, 50): 0, (8, 51): 0, (8, 52): 0, (8, 53): 6, (8, 54): 6, (8, 55): 0, (8, 56): 0, (8, 57): 0, (8, 58): 6, (8, 59): 6, (8, 60): 0, (8, 61): 0, (8, 62): 6, (8, 63): 6, (8, 64): 0, (8, 65): 0, (8, 66): 8, (8, 67): 6, (8, 68): 0, (8, 69): 0, (8, 70): 0, (8, 71): 8, (8, 72): 8, (8, 73): 2, (8, 74): 2, (8, 75): 2, (8, 76): 8, (9, 77): 6, (9, 78): 0, (9, 79): 0, (9, 80): 0, (9, 81): 6, (9, 82): 

In [36]:
dicc_PScs = {}
for index, row in df_relation_class_room.iterrows():
    llave = (int(row["ClassId"]),int(row["RoomId"]))
    dicc_PScs[llave] = int(row["RoomPenalty"]) 
print(dicc_PScs)

{(1, 18): 0, (15, 23): 0, (15, 8): 0, (15, 12): 0, (16, 23): 4, (16, 8): 0, (16, 12): 1, (19, 8): 0, (21, 22): 0, (22, 23): 4, (22, 8): 0, (22, 12): 1, (25, 23): 0, (25, 12): 0, (26, 9): 0, (26, 5): 0, (27, 22): 0, (28, 23): 0, (28, 8): 0, (28, 12): 0, (29, 16): 4, (29, 21): 0, (29, 22): 4, (29, 3): 0, (29, 13): 0, (30, 21): 0, (30, 3): 0, (30, 13): 0, (30, 25): 14, (30, 27): 10, (30, 7): 10, (30, 17): 10, (31, 16): 4, (31, 21): 4, (31, 22): 4, (31, 3): 4, (31, 13): 4, (31, 25): 4, (31, 27): 4, (31, 30): 0, (31, 6): 0, (31, 7): 1, (31, 17): 1, (32, 16): 0, (32, 22): 0, (32, 23): 4, (32, 8): 4, (32, 12): 4, (33, 16): 0, (33, 22): 0, (33, 23): 4, (33, 8): 4, (33, 12): 4, (34, 8): 0, (35, 16): 0, (35, 22): 0, (35, 23): 4, (35, 8): 4, (35, 12): 4, (36, 30): 0, (37, 16): 14, (37, 21): 14, (37, 22): 14, (37, 3): 14, (37, 13): 14, (37, 25): 14, (37, 27): 4, (37, 30): 0, (38, 27): 0, (39, 27): 0, (40, 16): 4, (40, 21): 0, (40, 22): 4, (40, 3): 0, (40, 13): 0, (40, 25): 4, (40, 27): 0, (40, 7):

In [37]:
##################### GUROBI #######################

In [38]:
from gurobipy import *
from random   import *
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [39]:
dicc_clases = {}
dicc_salas = {}
dicc_estudiantes = {}
dicc_tiempos = {}
dicc_combinaciones = {}
for e, clase in enumerate(dicc_classes):
    dicc_clases[e+1]= int(clase)

for index, estudiante in df_students.iterrows():
    dicc_estudiantes[index+1] = int(estudiante["StudentId"])

for index, sala in df_rooms.iterrows():
    dicc_salas[index+1] = int(sala["RoomId"])

c = 0
for clase in dicc_TDc:
    for tiempo in dicc_TDc[clase]:
        if tiempo not in dicc_tiempos.values():
            dicc_tiempos[c] = int(tiempo)
            c += 1

dicc_classes_times = {}
for index, row in df_relation_class_time.iterrows():
    if row["ClassId"] not in dicc_classes_times.keys():
        dicc_classes_times[row["ClassId"]] = []
    lista = [row["TimeDays"],row["TimeStart"],row["TimeLength"],row["TimeWeeks"],row["TimePenalty"]]
    dicc_classes_times[row["ClassId"]].append(lista)
    lista = []

dicc_configuraciones = {}
for index,row in df_configs.iterrows():
    dicc_configuraciones[int(row["ConfigId"])] = [int(x) for x in row["SubpartIdList"]]
print(dicc_configuraciones)

{1: [1], 2: [2], 3: [3], 4: [4], 5: [5, 6], 6: [7], 7: [8], 8: [9], 9: [10, 11], 10: [12], 11: [13], 12: [14], 13: [15], 14: [16], 15: [17], 16: [18, 19], 17: [20, 21], 18: [22], 19: [23], 20: [24], 21: [25], 22: [26], 23: [27], 24: [28], 25: [29], 26: [30], 27: [31], 28: [32], 29: [33], 30: [34, 35], 31: [36], 32: [37], 33: [38], 34: [39], 35: [40], 36: [41], 37: [42, 43], 38: [44, 45], 39: [46, 47, 48], 40: [49], 41: [50], 42: [51, 52], 43: [53], 44: [54], 45: [55, 56], 46: [57], 47: [58, 59], 48: [60, 61, 62], 49: [63], 50: [64], 51: [65], 52: [66, 67], 53: [68, 69], 54: [70], 55: [71], 56: [72], 57: [73], 58: [74], 59: [75], 60: [76], 61: [77, 78, 79], 62: [80, 81], 63: [82, 83], 64: [84, 85], 65: [86, 87], 66: [88, 89], 67: [90, 91], 68: [92, 93], 69: [94, 95, 96], 70: [97, 98], 71: [99], 72: [100], 73: [101], 74: [102], 75: [103], 76: [104], 77: [105], 78: [106], 79: [107], 80: [108], 81: [109], 82: [110], 83: [111], 84: [112], 85: [113], 86: [114], 87: [115], 88: [116], 89: [117

In [40]:
c = 0
dicc_tiempos2 = {}
for clase in dicc_TDc2:
    for tiempo in dicc_TDc2[clase]:
        if tiempo not in dicc_tiempos2.values():
            dicc_tiempos2[c] = tiempo
            c += 1
print(dicc_tiempos2)

{0: ['0000100', '168', 46, '011110111111110', '0'], 1: ['1000000', '96', 46, '011111011111110', '0'], 2: ['0000100', '96', 46, '011110111111110', '0'], 3: ['1000000', '108', 46, '011111011111110', '0'], 4: ['0000100', '108', 46, '011110111111110', '0'], 5: ['1000000', '120', 46, '011111011111110', '0'], 6: ['0000100', '120', 46, '011110111111110', '0'], 7: ['1000000', '132', 46, '011111011111110', '0'], 8: ['0000100', '132', 46, '011110111111110', '2'], 9: ['1000000', '144', 46, '011111011111110', '0'], 10: ['0000100', '144', 46, '011110111111110', '2'], 11: ['1000000', '156', 46, '011111011111110', '0'], 12: ['0000100', '156', 46, '011110111111110', '2'], 13: ['1000000', '168', 46, '011111011111110', '0'], 14: ['0000100', '168', 46, '011110111111110', '2'], 15: ['1000000', '180', 46, '011111011111110', '2'], 16: ['0000100', '180', 46, '011110111111110', '2'], 17: ['1000000', '192', 46, '011111011111110', '2'], 18: ['0000100', '192', 46, '011110111111110', '2'], 19: ['0100000', '168', 

In [41]:
## De Cantidad de cursos pedidos por estudiante E
dicc_De = {}
for index,row in df_students.iterrows():
    dicc_De[int(row["StudentId"])] = len(row["CoursesList"])
print(dicc_De)

{1: 5, 2: 5, 3: 3, 4: 5, 5: 5, 6: 5, 7: 4, 8: 1, 9: 1, 10: 3, 11: 7, 12: 8, 13: 8, 14: 8, 15: 8, 16: 8, 17: 8, 18: 8, 19: 8, 20: 8, 21: 8, 22: 8, 23: 8, 24: 7, 25: 7, 26: 1, 27: 4, 28: 5, 29: 5, 30: 4, 31: 5, 32: 5, 33: 5, 34: 5, 35: 5, 36: 5, 37: 5, 38: 6, 39: 5, 40: 3, 41: 8, 42: 2, 43: 2, 44: 4, 45: 5, 46: 8, 47: 1, 48: 8, 49: 10, 50: 5, 51: 10, 52: 11, 53: 5, 54: 10, 55: 8, 56: 6, 57: 6, 58: 11, 59: 14, 60: 4, 61: 9, 62: 8, 63: 12, 64: 9, 65: 3, 66: 8, 67: 15, 68: 8, 69: 5, 70: 4, 71: 9, 72: 9, 73: 6, 74: 4, 75: 1, 76: 1, 77: 7, 78: 4, 79: 6, 80: 12, 81: 3, 82: 4, 83: 7, 84: 7, 85: 5, 86: 15, 87: 8, 88: 10, 89: 16, 90: 6, 91: 8, 92: 7, 93: 2, 94: 6, 95: 11, 96: 2, 97: 4, 98: 7, 99: 8, 100: 5, 101: 8, 102: 1, 103: 3, 104: 8, 105: 5, 106: 8, 107: 10, 108: 7, 109: 5, 110: 7, 111: 4, 112: 6, 113: 5, 114: 11, 115: 10, 116: 2, 117: 2, 118: 12, 119: 9, 120: 10, 121: 6, 122: 8, 123: 5, 124: 8, 125: 3, 126: 3, 127: 9, 128: 7, 129: 1, 130: 1, 131: 7, 132: 8, 133: 2, 134: 2, 135: 12, 136: 7, 

In [42]:
dicc_PD1i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameAttendees":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD1i[int(row["DistributionId"])] = dp
        
dicc_PD3i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameRoom":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD3i[int(row["DistributionId"])] = dp
        
dicc_PD4i = {}
for index,row in df_distributions.iterrows():
    if row["DistributionType"] == "SameTime":
        if row["DistributionPenalty"] is None:
            dp = 0
        else:
            dp = int(row["DistributionPenalty"])
        dicc_PD4i[int(row["DistributionId"])] = dp
print(dicc_PD1i)

{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 137: 0, 138: 0, 139

In [43]:
## Cursos pedidos por estudiante E
dicc_Ce = {}
#for index,row in df_students.iterrows():
#       dicc_Ce[int(row["StudentId"])] = row["CoursesList"]
#print(dicc_Ce)

for index,row in df_students.iterrows():
    dicc_Ce[int(row["StudentId"])] = []
    for i in row["CoursesList"]:
        dicc_Ce[int(row["StudentId"])].append(int(i))
        #print(i)
#print(dicc_Ce)

#Cantidad de subpartes de una configuración
dicc_CS = {}
for index, col in df_configs.iterrows():
    dicc_CS[int(col["ConfigId"])] = len(col["SubpartIdList"])
#print(dicc_CS)

#Cantidad de subpartes que pide un estudiante E
dicc_Se = {}
for i in dicc_Ce.keys():
    dicc_Se[i]=0
    for j in dicc_Ce[i]:
       
        cant = dicc_CS[j]
        dicc_Se[i]+= cant
        #print(j)
print(dicc_Se)

{1: 10, 2: 10, 3: 6, 4: 10, 5: 10, 6: 10, 7: 9, 8: 2, 9: 2, 10: 5, 11: 11, 12: 12, 13: 12, 14: 12, 15: 13, 16: 13, 17: 12, 18: 13, 19: 12, 20: 13, 21: 12, 22: 12, 23: 12, 24: 11, 25: 11, 26: 1, 27: 7, 28: 9, 29: 9, 30: 7, 31: 9, 32: 9, 33: 9, 34: 8, 35: 8, 36: 8, 37: 9, 38: 7, 39: 9, 40: 5, 41: 10, 42: 4, 43: 8, 44: 6, 45: 8, 46: 10, 47: 1, 48: 12, 49: 15, 50: 5, 51: 14, 52: 17, 53: 9, 54: 23, 55: 14, 56: 10, 57: 10, 58: 18, 59: 19, 60: 5, 61: 14, 62: 12, 63: 21, 64: 15, 65: 3, 66: 13, 67: 20, 68: 14, 69: 8, 70: 7, 71: 15, 72: 11, 73: 8, 74: 4, 75: 2, 76: 2, 77: 10, 78: 5, 79: 11, 80: 18, 81: 6, 82: 6, 83: 12, 84: 12, 85: 9, 86: 21, 87: 11, 88: 14, 89: 22, 90: 11, 91: 9, 92: 11, 93: 4, 94: 9, 95: 17, 96: 7, 97: 5, 98: 16, 99: 11, 100: 9, 101: 13, 102: 2, 103: 5, 104: 13, 105: 6, 106: 13, 107: 14, 108: 11, 109: 8, 110: 10, 111: 6, 112: 10, 113: 12, 114: 16, 115: 25, 116: 3, 117: 3, 118: 14, 119: 14, 120: 14, 121: 8, 122: 11, 123: 7, 124: 12, 125: 6, 126: 3, 127: 14, 128: 12, 129: 2, 130

In [44]:
import math
#Agrupar capacidad de cursos

#print("Límite de las clases | Cantidad de clases con el límite")
#display(df_classes.groupby("ClassLimit").count()["ClassId"])

dicc_cursos = {}
#display(df_courses)
for index, row in df_courses.iterrows():
    dicc_cursos[row["CourseId"]] = row["ConfigIdList"]
#print(dicc_cursos)      
dicc_configs = {}
#display(df_configs)
for index, row in df_configs.iterrows():
    dicc_configs[row["ConfigId"]] = row["SubpartIdList"]
#print(dicc_configs)
dicc_subparts = {}
#display(df_subparts)
for index, row in df_subparts.iterrows():
    dicc_subparts[row["SubpartId"]] = row["ClassIdList"]
#print(dicc_subparts)
dicc_classes = {}
#display(df_classes)
for index, row in df_classes.iterrows():
    dicc_classes[row["ClassId"]] = row["ClassLimit"]
#print(dicc_classes)

dicc_curso_clase = {}
for p,i in dicc_cursos.items():
    for j in i:
        if j in dicc_configs.keys():
            for k in dicc_configs[j]:
                if k in dicc_subparts.keys():
                    for l in dicc_subparts[k]:
                        if int(p) not in dicc_curso_clase:
                            dicc_curso_clase[int(p)] = []
                        dicc_curso_clase[int(p)].append(int(l))
print(dicc_curso_clase)

{1: [1], 2: [2], 3: [3], 4: [4], 5: [5, 6], 6: [7], 7: [8, 9], 8: [10, 11, 12, 13, 14], 9: [15, 16], 10: [17], 11: [18], 12: [19], 13: [20], 14: [21], 15: [22], 16: [23, 24], 17: [25, 26], 18: [27], 19: [28], 20: [29], 21: [30], 22: [31], 23: [32], 24: [33], 25: [34], 26: [35], 27: [36], 28: [37], 29: [38], 30: [39, 40], 31: [41], 32: [42], 33: [43], 34: [44], 35: [45], 36: [46], 37: [47, 48, 49], 38: [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73], 39: [74, 75, 76, 77, 78, 79], 40: [80, 81], 41: [82], 42: [83, 84, 85], 43: [86], 44: [87], 45: [88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106], 46: [107], 47: [108, 109], 48: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121], 49: [122], 50: [123], 51: [124], 52: [125, 126, 127, 128], 53: [129, 130, 131], 54: [132], 55: [133], 56: [134], 57: [135], 58: [136], 59: [137], 60: [138], 61: [139, 140, 141], 62: [142, 143], 63: [144, 145, 146, 147, 148,

In [45]:
dicc_CEe = {}
for i in dicc_Ce:
    for j in dicc_Ce[i]:
        if i not in dicc_CEe.keys():
            dicc_CEe[i] = []
        for m in dicc_curso_clase[j]:
            dicc_CEe[i].append(m)
print(dicc_CEe)

{1: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 125, 126, 127, 128, 193, 194, 195, 196, 197, 198, 199, 325, 144, 145, 146, 147, 148, 149, 150, 151, 152], 2: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 125, 126, 127, 128, 193, 194, 195, 196, 197, 198, 199, 325, 144, 145, 146, 147, 148, 149, 150, 151, 152], 3: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 193, 194, 195, 196, 197, 198, 199, 325], 4: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 125, 126, 127, 128, 193, 194, 195, 196, 197, 198, 199, 325, 144, 145, 146, 147, 148, 149, 150, 151, 152], 5: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 125, 126, 127, 128, 193, 194, 195, 196, 197, 198, 199, 325, 144, 145, 146, 147, 148, 149, 150, 151, 152], 6: [110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 125, 126, 127, 128, 193, 194, 195, 196, 197, 198, 199, 325, 144, 145, 146, 147, 148, 149, 150, 151, 152], 7: [110, 111, 112, 113, 114, 115, 116, 117, 

In [46]:
# 253 clases con capacidad menor o igual a 20
# 245 clases con capacidad mayor a 20 y menor o igual a 50
# 77 clases con capacidad mayor a 50

dicc_cap = {}
for index,row in df_classes.iterrows():
    dicc_cap[int(row["ClassId"])] = int(row["ClassLimit"])
#print(dicc_cap)

clases_grandes_estudiante = {}
clases_medianas_estudiante = {}
clases_chicas_estudiante = {}

for i in dicc_CEe.keys():
    for j in dicc_CEe[i]:
                
        if dicc_cap[j]>50:
            if i not in clases_grandes_estudiante.keys():
                clases_grandes_estudiante[i]=[j]
            else:
                clases_grandes_estudiante[i].append(j)
                
        elif dicc_cap[j]<=50 and dicc_cap[j]>20:
            if i not in clases_medianas_estudiante.keys():
                clases_medianas_estudiante[i]=[j]
            else:
                clases_medianas_estudiante[i].append(j)
        elif dicc_cap[j]<=20:
            if i not in clases_chicas_estudiante.keys():
                clases_chicas_estudiante[i]=[j]
            else:
                clases_chicas_estudiante[i].append(j)

print("grandes:",clases_grandes_estudiante)
#print("medianas:",clases_medianas_estudiante)
#print("chicas:",clases_chicas_estudiante)

grandes: {1: [110, 111, 125, 193, 144], 2: [110, 111, 125, 193, 144], 3: [110, 111, 193], 4: [110, 111, 125, 193, 144], 5: [110, 111, 125, 193, 144], 6: [110, 111, 125, 193, 144], 7: [110, 111, 125, 193, 144], 8: [144], 9: [125], 10: [125, 144], 11: [45, 394, 129, 470], 12: [45, 394, 129, 470], 13: [211, 45, 394, 129], 14: [45, 394, 129], 15: [45, 394, 129], 16: [45, 394, 129, 470], 17: [45, 394, 129], 18: [211, 45, 394, 129], 19: [45, 394, 129, 470], 20: [211, 45, 394, 129], 21: [211, 45, 394, 129], 22: [211, 45, 394, 129], 23: [45, 394, 129, 470], 24: [45, 394, 129], 25: [45, 394, 129], 26: [34], 27: [212, 129], 28: [394, 129], 29: [394, 129], 30: [394, 129], 31: [394, 129], 32: [394, 129], 33: [394, 129], 34: [394, 129], 35: [394, 129], 36: [394, 129], 37: [394, 129], 38: [45, 193], 39: [110, 111, 211, 193], 40: [175, 293, 306], 41: [212, 175, 449, 213, 450, 226], 42: [175, 50], 43: [410, 411, 412], 44: [476, 129], 45: [193, 144], 46: [211, 212, 125, 193, 217, 34, 35], 47: [107], 48

In [47]:
# Asignacion clases grandes
timetabling = gp.Model("Asignacion de cursos a horarios y salas")

#Conjuntos
C = dicc_clases_grandes.keys()
F = dicc_configuraciones.keys()
S = dicc_salas.keys()
T = dicc_tiempos.keys()
E = clases_grandes_estudiante.keys()
I = lista_I

#subconjuntos
SDisp_c = dicc_SDispc2
TD_c = dicc_TDc
DF_e = dicc_DFe
UT_s = dicc_UTs
SPF_f = dicc_SPFf
SA_c = dicc_SAc
SA_c_B = dicc_SAc_B
SS_c = dicc_SSc
SS_c_B = dicc_SSc_B
SR_c = dicc_SRc
SR_c_B = dicc_SRc2
PR_c = dicc_PRc
PR_c_B = dicc_PRc_B
SD_c = dicc_SDc
SD_c_B = dicc_SDc_B
D_j = dicc_Dj
I_j = dicc_Ij
ST_c = dicc_STc
ST_c_B = dicc_STc_B
De = dicc_De
Se = dicc_Se
CEe = clases_grandes_estudiante

#parámetros
CP_c = dicc_CPc
QC_c = dicc_QCc
QS_s = dicc_QSs
D_c = dicc_Dc
TS_c = dicc_TSc
PS_cs = dicc_PScs
PT_ct = dicc_PTct
PD_1i = dicc_PD1i
#PD_2i
PD_3i = dicc_PD3i
PD_4i = dicc_PD4i
#PD_5i
#PD_6i
#PE_
alfa = int(df_optimization["Time"])
beta = int(df_optimization["Room"])
gamma = int(df_optimization["Distribution"])
delta = int(df_optimization["Student"])



x = {} 
for c in dicc_clases_grandes.keys():
    for t in dicc_tiempos.keys():
        x[c,t] = timetabling.addVar(vtype=gp.GRB.BINARY, name='x_'+str(c)+','+str(t))

y = {}
for c in dicc_clases_grandes.keys():
    for s in dicc_salas.keys():
        y[c,s] = timetabling.addVar(vtype=gp.GRB.BINARY, name='y_'+str(c)+','+str(s))

z = {}
for e in clases_grandes_estudiante.keys():
    for c in dicc_clases_grandes.keys():
        z[e,c] = timetabling.addVar(vtype=gp.GRB.BINARY, name='z_'+str(e)+','+str(c))

w = {}
for e in clases_grandes_estudiante.keys():
    for f in dicc_configuraciones.keys():
        w[e,f] = timetabling.addVar(vtype=gp.GRB.BINARY, name='w_'+str(e)+','+str(f))
        
xs = {}
for c in dicc_clases_grandes.keys():
    for t in dicc_tiempos.keys():
        xs[c,t] = timetabling.addVar(vtype=gp.GRB.BINARY, name='xs_'+str(c)+','+str(t))

aux1 = {}
for i in I_j[1]:
    aux1[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux1_"+str(i))
    
aux2 = {}
for i in I_j[2]:
    aux2[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux2_"+str(i))
    
aux3 = {}
for i in I_j[3]:
    aux3[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux3_"+str(i))
    
aux4 = {}
for i in I_j[4]:
    aux4[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux4_"+str(i))
    
aux5 = {}
for i in I_j[5]:
    aux5[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux5_"+str(i))
    
aux6 = {}
for i in I_j[6]:
    aux6[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux6_"+str(i))
        
clash = {}
for c1 in dicc_clases_grandes.keys():
    for c2 in dicc_clases_grandes.keys():
        clash[c1, c2] = timetabling.addVar(vtype=gp.GRB.BINARY, name="clash_"+str(c1)+","+str(c2))


        
#Función Objetivo
timetabling.setObjective(alfa*gp.quicksum(x[c,t]*PT_ct[(c,t)] for c in C for t in TD_c[c])+beta*gp.quicksum(y[c,s]*PS_cs[(c,s)] for c in C for s in SDisp_c[c])+gamma*(gp.quicksum(aux1[i]*PD_1i[i] for i in I_j[1])+gp.quicksum(aux3[i]*PD_3i[i] for i in I_j[3])+gp.quicksum(aux4[i]*PD_4i[i] for i in I_j[4]))+delta*gp.quicksum(clash[c1,c2] for c1 in C for c2 in C),GRB.MINIMIZE)

#Restricciones
#A cada clase se le debe ser asignado un tiempo
timetabling.addConstrs(((gp.quicksum(x[c,t] for t in TD_c[c]) == 1) for c in C), name = "R1")
#Cada clase debe ser asignada a una sala (donde sea aplicable)
timetabling.addConstrs(((gp.quicksum(y[c,s] for s in SDisp_c[c]) == 1) for c in C), name = "R2")
#Cada estudiante debe ser asignado a una configuración de curso para cada curso que demanda
timetabling.addConstrs(((w[e,f] == 1) for e in E for f in DF_e[e]), name = "R3")
#hola
#timetabling.addConstrs(((gp.quicksum(w[e,f] for f in DF_e[e]) <= De[e]) for e in E), name = "R3b")
#Cada estudiante debe ser asignado a todas las clases que requiere
timetabling.addConstrs(((z[e,c] == 1) for e in E for c in CEe[e]), name = "E")
#La capacidad de cada clase en términos del número de estudiantes debe ser satisfecha
#timetabling.addConstrs(((gp.quicksum(z[e,c] for e in E) <= QC_c[c]*x[c,t]) for c in CEe[e] for t in TD_c[c]), name = "R4")
#Una sala no puede ser usada cuando no está disponible
#timetabling.addConstrs(((y[c,s] + x[c,t] <= 1) for c in C for s in SDisp_c[c] for t in UT_s[s]), name = "R5")
#Dos clases no pueden ser asignadas en un mismo tiempo en la misma sala
timetabling.addConstrs(((x[c1,t1] + y[c1,s1] + x[c1,t2] + y[c2,s2] <= 3) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1 != c2 if dicc_tiempos2[c1][0] == dicc_tiempos2[c2][0] if dicc_tiempos2[c1][1] == dicc_tiempos2[c2][1] if s1 == s2), name = "R6")
#Si la clase 1 y la clase 2 se dictan el mismo dia en el mismo start, se activa variable binaria
timetabling.addConstrs(((z[e,c1] + z[e,c2] <= 1 + clash[c1,c2]) for e in E for c1 in CEe[e] for c2 in CEe[e] if c1 != c2 if dicc_tiempos2[c1][0] == dicc_tiempos2[c2][0] if dicc_tiempos2[c1][1] == dicc_tiempos2[c2][1]), name = "Clash")
#SAME ATTENDEES
timetabling.addConstrs(((x[c1,t1] + x[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in SA_c[c2] or c2 in SA_c[c1]), name = "R7")
#SAME START
#timetabling.addConstrs(((xs[c1,t1] + xs[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if dicc_tiempos2[c1][1]==dicc_tiempos2[c2][1] if c1 in SS_c[c2] or c2 in SS_c[c1]), name = "R8")
#SAME ROOMS
timetabling.addConstrs(((y[c1,s1] + y[c2,s2] == 2) for c1 in C for c2 in C for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1!=c2 if s1==s2 if c1 in SR_c[c2] or c2 in SR_c[c1]), name = "R9")
#SAME TIME
timetabling.addConstrs(((x[c1,t1] + x[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in ST_c[c2] or c2 in ST_c[c1]), name = "R10")
#SAME DAYS
#timetabling.addConstrs(((x[c1,t1] <= gp.quicksum(xs[c2,t2] for t2 in TD_c[c2])) for c1 in C for c2 in C for t1 in TD_c[c1] if c1!=c2 if dicc_tiempos2[c1][0]==dicc_tiempos2[c2][0] if c1 in SD_c[c2] or c2 in SD_c[c1]), name = "R11")
#timetabling.addConstrs(((xs[c1,t1] == xs[c2,t2+D_c[c2]+1]) for c1 in PR_c[c1] for c2 in PR_c[c2] for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2), name = "R13")


#Restricciones blandas
timetabling.addConstrs((((x[c1,t1] - x[c2,t2])*(x[c1,t1] - x[c2,t2]) == aux1[i]) for i in I_j[1] for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if t1 == t2 if c1 in SA_c_B[c2] or c2 in SA_c_B[c1]), name = "SameAttendees B")

#timetabling.addConstrs((((xs[c1,t1]-xs[c2,t2])*(xs[c1,t1]-xs[c2,t2]) == aux2[i]) for i in I_j[2] for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if dicc_tiempos2[t1][1]==dicc_tiempos2[t2][1]), name = "SameStartB")

timetabling.addConstrs((((y[c1,s1] - y[c2,s2])*(y[c1,s1] - y[c2,s2]) == aux3[i]) for i in I_j[3] for c1 in C for c2 in C for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1 != c2 if s1 == s2 if c1 in SR_c_B[c2] or c2 in SR_c_B[c1]), name = "SameRoomsB")

timetabling.addConstrs((((x[c1,t1] - x[c2,t2])*(x[c1,t1] - x[c2,t2]) == aux4[i]) for i in I_j[4] for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in ST_c_B[c2] or c2 in ST_c_B[c1]), name = "SameTimeB")

#timetabling.addConstrs(((xs[c1,t1]-gp.quicksum(xs[c2,t2] for t2 in TD_c[c2]) <= aux5[i]) for i in I_j[5] for c1 in C for c2 in C for t1 in TD_c[c1] if c1!=c2 if dicc_tiempos2[c1][0]==dicc_tiempos2[c2][0]), name = "SameDaysB")

#timetabling.addConstrs((xs[c1,t1]-xs[c2,t1+dc+1] == Aux8[8,i] for c1 in PR_c[c1] for c2 in PR_c[c2] for t1 in TD_c[c1] if c1!=c2), name = "PrecedenceB")

## Relación entre variables

timetabling.addConstrs((x[c,t1] <= xs[c,t2] for c in C for t1 in TD_c[c] for t2 in TD_c[c] if t1 == t2), name ="RelacionX-XSn1")

timetabling.addConstrs(((xs[c,t] <= gp.quicksum(x[c,t] for t in TD_c[c])) for c in C), name = "RelacionX-XSn2")

#timetabling.addConstrs(((x[c,t] == y[c,s]) for c in C for t in TD_c[c] for s in SDisp_c[c]), name = "RelacionX-Y")
timetabling.addConstrs(((quicksum(z[e,c] for c in C) >= quicksum(w[e,f] for f in F)) for e in E), name = "RelacionZ-W")


timetabling.optimize()


Using license file C:\Users\Perro\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 90101 rows, 1937428 columns and 756163 nonzeros
Model fingerprint: 0xcd29ab6c
Variable types: 0 continuous, 1937428 integer (1937428 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 1570.0000000
Presolve removed 90101 rows and 1937428 columns
Presolve time: 0.54s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.90 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 450 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.500000000000e+02, best bound 4.500000000000e+02, gap 0.0000%


In [48]:
#for v in timetabling.getVars():
#    print('%s %g' % (v.varName, v.x))

In [49]:
#timetabling.write("file_infeasible.lp")
#timetabling.write("file_infeasible.mps")
#timetabling.write("resultados_clases_muy_grandes.sol")

In [50]:
import csv

varInfo = [(v.varName, int(v.X)) for v in timetabling.getVars() if v.X > 0]

# Write to csv
with open('solucionearly1.csv', 'w', newline="") as myfile:
    wr = csv.writer(myfile, delimiter = ";", quoting=csv.QUOTE_ALL)
    wr.writerows(varInfo)

In [59]:
f = open("solucionearly1.csv", "r")
reader = csv.reader(f, delimiter=";")

listax = []
listay = []
listaz = []
listaw = []
listaxs = []
listaaux1 = []
listaaux2 = []
listaaux3 = []
listaaux4 = []
listaaux5 = []
listaaux6 = []
listaclash = []

for variable in reader:
    if variable[0][0] == "y":
        listay.append(variable[0])
    if variable[0][0] == "x" and variable[0][1] == "_":
        listax.append(variable[0])
    if variable[0][0] == "z":
        listaz.append(variable[0])
    if variable[0][0] == "w":
        listaw.append(variable[0])
    if variable[0][0] == "x" and variable[0][1] == "s":
        listaxs.append(variable[0])
    if variable[0][0] == "a" and variable[0][3] == "1":
        listaaux1.append(variable[0])
    if variable[0][0] == "a" and variable[0][3] == "2":
        listaaux2.append(variable[0])
    if variable[0][0] == "a" and variable[0][3] == "3":
        listaaux3.append(variable[0])
    if variable[0][0] == "a" and variable[0][3] == "4":
        listaaux4.append(variable[0])
    if variable[0][0] == "a" and variable[0][3] == "5":
        listaaux5.append(variable[0])
    if variable[0][0] == "a" and variable[0][3] == "6":
        listaaux6.append(variable[0])
    if variable[0][0] == "c":
        listaclash.append(variable[0])


#print(listax)
#print(listay)
#print(listaz)
#print(listaw)
#print(listaxs)
#print(listaaux1)
#print(listaaux2)
#print(listaaux3)
#print(listaaux4)
#print(listaaux5)
#print(listaaux6)
#print(listaclash)

listasubindicesy = []
for i in listay:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = (int(gato[0]),int(gato[1]))
            listasubindicesy.append(perrogato)

listasubindicesx = []
for i in listax:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = (int(gato[0]),int(gato[1]))
            listasubindicesx.append(perrogato)

listasubindicesz = []
for i in listaz:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = (int(gato[0]),int(gato[1]))
            listasubindicesz.append(perrogato)

listasubindicesw = []
for i in listaw:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = (int(gato[0]),int(gato[1]))
            listasubindicesw.append(perrogato)

listasubindicesxs = []
for i in listaxs:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = (int(gato[0]),int(gato[1]))
            listasubindicesxs.append(perrogato)

listasubindicesaux1 = []
for i in listaaux1:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            #gato = perro.split(",")
            #print(gato)
            perrogato = (int(perro))
            listasubindicesaux1.append(perrogato)

listasubindicesaux2 = []
for i in listaaux2:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            #gato = perro.split(",")
            #print(gato)
            perrogato = (int(perro))
            listasubindicesaux2.append(perrogato)

listasubindicesaux3 = []
for i in listaaux3:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            #gato = perro.split(",")
            #print(gato)
            perrogato = (int(perro))
            listasubindicesaux3.append(perrogato)
            
listasubindicesaux4 = []
for i in listaaux4:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            #gato = perro.split(",")
            #print(gato)
            perrogato = (int(perro))
            listasubindicesaux4.append(perrogato)
            
listasubindicesaux5 = []
for i in listaaux5:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            #gato = perro.split(",")
            #print(gato)
            perrogato = (int(perro))
            listasubindicesaux5.append(perrogato)
            
listasubindicesaux6 = []
for i in listaaux6:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            #gato = perro.split(",")
            #print(gato)
            perrogato = (int(perro))
            listasubindicesaux6.append(perrogato)
            
listasubindicesclash = []
for i in listaclash:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = (int(gato[0]),int(gato[1]))
            listasubindicesclash.append(perrogato)

#print(listasubindicesx)
#print(listasubindicesy)
#print(listasubindicesw)
#print(listasubindicesz)
#print(listasubindicesaux1)
#print(listasubindicesaux2)
#print(listasubindicesaux3)
#print(listasubindicesaux4)
#print(listasubindicesaux5)
#print(listasubindicesaux6)
#print(listasubindicesclash)

#for i in listasubindicesx:
#    print(i[0],i[1])

f.close()

In [ ]:
# Asignacion clases medianas
timetabling2 = gp.Model("Asignacion de cursos a horarios y salas")

#Conjuntos
C = dicc_clases_medianas.keys()
F = dicc_configuraciones.keys()
S = dicc_salas.keys()
T = dicc_tiempos.keys()
E = clases_medianas_estudiante.keys()
I = lista_I

#subconjuntos
SDisp_c = dicc_SDispc2
TD_c = dicc_TDc
DF_e = dicc_DFe
UT_s = dicc_UTs
SPF_f = dicc_SPFf
SA_c = dicc_SAc
SA_c_B = dicc_SAc_B
SS_c = dicc_SSc
SS_c_B = dicc_SSc_B
SR_c = dicc_SRc
SR_c_B = dicc_SRc2
PR_c = dicc_PRc
PR_c_B = dicc_PRc_B
SD_c = dicc_SDc
SD_c_B = dicc_SDc_B
D_j = dicc_Dj
I_j = dicc_Ij
ST_c = dicc_STc
ST_c_B = dicc_STc_B
De = dicc_De
Se = dicc_Se
CEe = clases_medianas_estudiante

#parámetros
CP_c = dicc_CPc
QC_c = dicc_QCc
QS_s = dicc_QSs
D_c = dicc_Dc
TS_c = dicc_TSc
PS_cs = dicc_PScs
PT_ct = dicc_PTct
PD_1i = dicc_PD1i
#PD_2i
PD_3i = dicc_PD3i
PD_4i = dicc_PD4i
#PD_5i
#PD_6i
#PE_
alfa = int(df_optimization["Time"])
beta = int(df_optimization["Room"])
gamma = int(df_optimization["Distribution"])
delta = int(df_optimization["Student"])



x = {} 
for c in dicc_clases_medianas.keys():
    for t in dicc_tiempos.keys():
        x[(c,t)] = timetabling.addVar(vtype=gp.GRB.BINARY, name='x_'+str(c)+','+str(t))

y = {}
for c in dicc_clases_medianas.keys():
    for s in dicc_salas.keys():
        y[(c,s)] = timetabling.addVar(vtype=gp.GRB.BINARY, name='y_'+str(c)+','+str(s))

z = {}
for e in clases_medianas_estudiante.keys():
    for c in dicc_clases_medianas.keys():
        z[(e,c)] = timetabling.addVar(vtype=gp.GRB.BINARY, name='z_'+str(e)+','+str(c))

w = {}
for e in clases_medianas_estudiante.keys():
    for f in dicc_configuraciones.keys():
        w[(e,f)] = timetabling.addVar(vtype=gp.GRB.BINARY, name='w_'+str(e)+','+str(f))
        
xs = {}
for c in dicc_clases_medianas.keys():
    for t in dicc_tiempos.keys():
        xs[(c,t)] = timetabling.addVar(vtype=gp.GRB.BINARY, name='xs_'+str(c)+','+str(t))

aux1 = {}
for i in I_j[1]:
    aux1[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux1_"+str(i))
    
aux2 = {}
for i in I_j[2]:
    aux2[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux2_"+str(i))
    
aux3 = {}
for i in I_j[3]:
    aux3[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux3_"+str(i))
    
aux4 = {}
for i in I_j[4]:
    aux4[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux4_"+str(i))
    
aux5 = {}
for i in I_j[5]:
    aux5[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux5_"+str(i))
    
aux6 = {}
for i in I_j[6]:
    aux6[i] = timetabling.addVar(vtype=gp.GRB.BINARY, name="aux6_"+str(i))
        
clash = {}
for c1 in dicc_clases_medianas.keys():
    for c2 in dicc_clases_medianas.keys():
        clash[(c1, c2)] = timetabling.addVar(vtype=gp.GRB.BINARY, name="clash_"+str(c1)+","+str(c2))

for i in listasubindicesx:
    x[i[0],i[1]] = timetabling.addVar(lb=1, ub=1)
    
for i in listasubindicesy:
    y[i[0],i[1]] = timetabling.addVar(lb=1, ub=1)
    
for i in listasubindicesw:
    w[i[0],i[1]] = timetabling.addVar(lb=1, ub=1)
    
for i in listasubindicesz:
    z[i[0],i[1]] = timetabling.addVar(lb=1, ub=1)

for i in listasubindicesclash:
    clash[i[0],i[1]] = timetabling.addVar(lb=1, ub=1)

        
#Función Objetivo
timetabling.setObjective(alfa*gp.quicksum(x[c,t]*PT_ct[(c,t)] for c in C for t in TD_c[c])+beta*gp.quicksum(y[c,s]*PS_cs[(c,s)] for c in C for s in SDisp_c[c])+gamma*(gp.quicksum(aux1[i]*PD_1i[i] for i in I_j[1])+gp.quicksum(aux3[i]*PD_3i[i] for i in I_j[3])+gp.quicksum(aux4[i]*PD_4i[i] for i in I_j[4]))+delta*gp.quicksum(clash[c1,c2] for c1 in C for c2 in C),GRB.MINIMIZE)

#Restricciones
#A cada clase se le debe ser asignado un tiempo
timetabling.addConstrs(((gp.quicksum(x[c,t] for t in TD_c[c]) == 1) for c in C), name = "R1")
#Cada clase debe ser asignada a una sala (donde sea aplicable)
timetabling.addConstrs(((gp.quicksum(y[c,s] for s in SDisp_c[c]) == 1) for c in C), name = "R2")
#Cada estudiante debe ser asignado a una configuración de curso para cada curso que demanda
timetabling.addConstrs(((w[e,f] == 1) for e in E for f in DF_e[e]), name = "R3")
#hola
#timetabling.addConstrs(((gp.quicksum(w[e,f] for f in DF_e[e]) <= De[e]) for e in E), name = "R3b")
#Cada estudiante debe ser asignado a todas las clases que requiere
timetabling.addConstrs(((z[e,c] == 1) for e in E for c in CEe[e]), name = "E")
#La capacidad de cada clase en términos del número de estudiantes debe ser satisfecha
#timetabling.addConstrs(((gp.quicksum(z[e,c] for e in E) <= QC_c[c]*x[c,t]) for c in CEe[e] for t in TD_c[c]), name = "R4")
#Una sala no puede ser usada cuando no está disponible
#timetabling.addConstrs(((y[c,s] + x[c,t] <= 1) for c in C for s in SDisp_c[c] for t in UT_s[s]), name = "R5")
#Dos clases no pueden ser asignadas en un mismo tiempo en la misma sala
timetabling.addConstrs(((x[c1,t1] + y[c1,s1] + x[c1,t2] + y[c2,s2] <= 3) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1 != c2 if dicc_tiempos2[c1][0] == dicc_tiempos2[c2][0] if dicc_tiempos2[c1][1] == dicc_tiempos2[c2][1] if s1 == s2), name = "R6")
#Si la clase 1 y la clase 2 se dictan el mismo dia en el mismo start, se activa variable binaria
timetabling.addConstrs(((z[e,c1] + z[e,c2] <= 1 + clash[c1,c2]) for e in E for c1 in CEe[e] for c2 in CEe[e] if c1 != c2 if dicc_tiempos2[c1][0] == dicc_tiempos2[c2][0] if dicc_tiempos2[c1][1] == dicc_tiempos2[c2][1]), name = "Clash")
#SAME ATTENDEES
timetabling.addConstrs(((x[c1,t1] + x[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in SA_c[c2] or c2 in SA_c[c1]), name = "R7")
#SAME START
#timetabling.addConstrs(((xs[c1,t1] + xs[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if dicc_tiempos2[c1][1]==dicc_tiempos2[c2][1] if c1 in SS_c[c2] or c2 in SS_c[c1]), name = "R8")
#SAME ROOMS
timetabling.addConstrs(((y[c1,s1] + y[c2,s2] == 2) for c1 in C for c2 in C for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1!=c2 if s1==s2 if c1 in SR_c[c2] or c2 in SR_c[c1]), name = "R9")
#SAME TIME
timetabling.addConstrs(((x[c1,t1] + x[c2,t2] == 2) for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in ST_c[c2] or c2 in ST_c[c1]), name = "R10")
#SAME DAYS
#timetabling.addConstrs(((x[c1,t1] <= gp.quicksum(xs[c2,t2] for t2 in TD_c[c2])) for c1 in C for c2 in C for t1 in TD_c[c1] if c1!=c2 if dicc_tiempos2[c1][0]==dicc_tiempos2[c2][0] if c1 in SD_c[c2] or c2 in SD_c[c1]), name = "R11")
#timetabling.addConstrs(((xs[c1,t1] == xs[c2,t2+D_c[c2]+1]) for c1 in PR_c[c1] for c2 in PR_c[c2] for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2), name = "R13")


#Restricciones blandas
timetabling.addConstrs((((x[c1,t1] - x[c2,t2])*(x[c1,t1] - x[c2,t2]) == aux1[i]) for i in I_j[1] for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if t1 == t2 if c1 in SA_c_B[c2] or c2 in SA_c_B[c1]), name = "SameAttendees B")

#timetabling.addConstrs((((xs[c1,t1]-xs[c2,t2])*(xs[c1,t1]-xs[c2,t2]) == aux2[i]) for i in I_j[2] for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1 != c2 if dicc_tiempos2[t1][1]==dicc_tiempos2[t2][1]), name = "SameStartB")

timetabling.addConstrs((((y[c1,s1] - y[c2,s2])*(y[c1,s1] - y[c2,s2]) == aux3[i]) for i in I_j[3] for c1 in C for c2 in C for s1 in SDisp_c[c1] for s2 in SDisp_c[c2] if c1 != c2 if s1 == s2 if c1 in SR_c_B[c2] or c2 in SR_c_B[c1]), name = "SameRoomsB")

timetabling.addConstrs((((x[c1,t1] - x[c2,t2])*(x[c1,t1] - x[c2,t2]) == aux4[i]) for i in I_j[4] for c1 in C for c2 in C for t1 in TD_c[c1] for t2 in TD_c[c2] if c1!=c2 if t1==t2 if c1 in ST_c_B[c2] or c2 in ST_c_B[c1]), name = "SameTimeB")

#timetabling.addConstrs(((xs[c1,t1]-gp.quicksum(xs[c2,t2] for t2 in TD_c[c2]) <= aux5[i]) for i in I_j[5] for c1 in C for c2 in C for t1 in TD_c[c1] if c1!=c2 if dicc_tiempos2[c1][0]==dicc_tiempos2[c2][0]), name = "SameDaysB")

#timetabling.addConstrs((xs[c1,t1]-xs[c2,t1+dc+1] == Aux8[8,i] for c1 in PR_c[c1] for c2 in PR_c[c2] for t1 in TD_c[c1] if c1!=c2), name = "PrecedenceB")

## Relación entre variables

timetabling.addConstrs((x[c,t1] <= xs[c,t2] for c in C for t1 in TD_c[c] for t2 in TD_c[c] if t1 == t2), name ="RelacionX-XSn1")

timetabling.addConstrs(((xs[c,t] <= gp.quicksum(x[c,t] for t in TD_c[c])) for c in C), name = "RelacionX-XSn2")

#timetabling.addConstrs(((x[c,t] == y[c,s]) for c in C for t in TD_c[c] for s in SDisp_c[c]), name = "RelacionX-Y")
timetabling.addConstrs(((quicksum(z[e,c] for c in C) >= quicksum(w[e,f] for f in F)) for e in E), name = "RelacionZ-W")

timetabling.optimize()

In [ ]:
f = open("solucionearly.csv", "r")
reader = csv.reader(f, delimiter=";")

listax = []
listay = []

for variable in reader:
    if variable[0][0] == "y":
        listay.append(variable[0])
    if variable[0][0] == "x" and variable[0][1] == "_":
        listax.append(variable[0])

#print(listax)
#print(listay)
listasubindicesy = []
for i in listay:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = [int(gato[0]),int(gato[1])]
            listasubindicesy.append(perrogato)

#print(listasubindicesy)
listalimclase = []
listacapsala = []
listarazon = []
for i in listasubindicesy:
    limclase = QC_c[i[0]]
    capsala = QS_s[i[1]]
    razon = limclase/capsala
    listarazon.append(razon)

#print(listarazon)
suma = 0
cont = 0
for i in listarazon:
    suma += i
    cont += 1
porcentaje_ocupacion = (suma/cont)*100
print("porcentaje ocupacion:", porcentaje_ocupacion)

listasubindicesx = []
for i in listax:
    for j in range(len(i)):
        if i[j] == "_":
            perro = i[j+1:len(i)+1]
            #print(perro)
            gato = perro.split(",")
            #print(gato)
            perrogato = [int(gato[0]),int(gato[1])]
            listasubindicesx.append(perrogato)
#print(listasubindicesx)

listaclasessinrepetir = []
for i in listasubindicesx:
    if i[0] not in listaclasessinrepetir:
        listaclasessinrepetir.append(i[0])
#print(listaclasessinrepetir)
perro1 = len(listaclasessinrepetir)
perro2 = len(C)

#print(perro1)
#print(perro2)

porcentaje_clases_asignadas = (perro1/perro2)*100

print("porcentaje clases asignadas:", porcentaje_clases_asignadas)


f.close()

In [ ]:
import xml.etree.ElementTree as ET

root = ET.Element("problema", name="pu-cs-fal07", nrDays="7", slotsPerDay="288", nrWeeks="15")

solucion = ET.SubElement(root, "solution")

f = open("soluciontest.csv", "r")
reader = csv.reader(f, delimiter=";")

listax = []
listay = []
listaz = []
listaw = []
listaxs = []
listaaux1 = []
listaaux2 = []
listaaux3 = []
listaaux4 = []
listaaux5 = []
listaaux6 = []
listaclash = []

for line in reader:
    print(line)

f.close()
#ET.SubElement(root, "class", )


#tree = ET.ElementTree(root)
#tree.write("soluciontest.xml")